In [ ]:
import pandas as pd 
templates_folder = "../templates/"
bitrix_file_2025              = "bitrix_2025-04-01_2025-09-15.xlsx"
leads_dates_2025_by_program = pd.read_excel(templates_folder + bitrix_file_2025) #, parse_dates=[0], date_format="%d.%m.%Y  %hh:%mm:%ss")
leads_dates_2025_by_program['leads_dates'] = pd.to_datetime(leads_dates_2025_by_program['leads_dates'], errors='coerce', format="%d.%m.%Y  %hh:%mm:%ss")
# leads_dates_2025_by_program[col_programs_names].fillna("main_studyonline", inplace=True)
leads_dates_2025_by_program.info()

In [ ]:
# Drop columns with mostly missing data or almost constant values
missing_threshold = 0.95   # drop if >=95% NaN
same_threshold = 0.98      # drop if one value takes >=98% of non-null rows

missing_ratio = leads_dates_2025_by_program.isna().mean()

dominant_ratio = leads_dates_2025_by_program.apply(
    lambda s: s.value_counts(dropna=True, normalize=True).iloc[0] if s.notna().any() else 1.0
)

cols_to_drop = leads_dates_2025_by_program.columns[
    (missing_ratio >= missing_threshold) | (dominant_ratio >= same_threshold)
]

leads_dates_2025_by_program = leads_dates_2025_by_program.drop(columns=cols_to_drop)


print(cols_to_drop.tolist())
print(f"Dropped columns: {len(cols_to_drop)}")
print(f"Remaining columns: {leads_dates_2025_by_program.shape[1]}")

In [ ]:
print("\n".join(cols_to_drop.tolist()))

In [ ]:
leads_dates_2025_by_program.info()

In [ ]:
leads_dates_2025_by_program.to_excel(templates_folder + bitrix_file_2025, index=False)

# Stage 12
22.01.26

In [ ]:
%pip install requests json datetime

In [ ]:
import requests
import json
from my_secrets import secrets
# Базовый URL webhook
webhook_url = secrets["BITRIX_WEBHOOK_URL"]
# Функция для выполнения запросов
def bitrix_request(method, params=None):
    url = webhook_url + method
    if params is None:
        params = {}
    response = requests.post(url, json=params)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Ошибка: {response.status_code}")
        return None

In [ ]:
def get_contacts(limit=50):
    params = {
    "select": ["ID", "NAME", "PHONE", "EMAIL"],
    "start": 0
    }
    result = bitrix_request("crm.contact.list", params)
    if result and result.get("result"):
        contacts = result["result"]
        for contact in contacts:
            print(f"ID: {contact['ID']}, Имя: {contact['NAME']}")
    return contacts

# Получение контактов m
contacts = get_contacts()

## Stage 1

In [ ]:
import pandas as pd

In [ ]:
def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

In [ ]:
# Define names and folders
relative_folder = "../data/"

programs_file = "programs.xlsx"

bitrix_file = "bitrix.xls"

master_file = "asav.xlsx"

template_file = "template.xlsx"

In [ ]:
col_applications = """Кол-во регистраций в ЛК абитуриента
(РФ 1 и 2 приоритет)"""
col_contracts = """Договоры
(ПК) РФ"""
col_payments = """Оплаты
(ПК)"""
col_enrollments = "Зачисленные (ПК)"

In [ ]:
# cчитываем базу данных програм
df_online_programs = pd.read_excel(relative_folder + programs_file)
df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by="program").reset_index(drop=True)
df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].sort_values(by="program").reset_index(drop=True)


In [ ]:
%%time
df_master = pd.read_excel(relative_folder + master_file) #, skiprows=1)


In [ ]:
%%time
df_master = pd.read_excel(relative_folder + master_file, sheet_name="только онлайн", skiprows=1, usecols="L:DT")


In [ ]:

master_file_sheet_name = "только онлайн"
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение"
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении"

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({'program':master_applications.index, 'values':master_applications.values})

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({'program':master_contracts.index, 'values':master_contracts.values})

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({'program':master_payments.index, 'values':master_payments.values})

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({'program':master_enrollments.index, 'values':master_enrollments.values})


In [ ]:
# создаем дашборд по магистратурам добавляя туда программы из базы
df_master_dashboard = pd.read_excel(relative_folder + template_file)
df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
.fillna(0)
df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)


In [ ]:
# вставляем в дашборд показатели из АСАВ

df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, 'program', col_applications)
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, 'program', col_contracts)
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, 'program', col_payments)
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, 'program', col_enrollments)

# df_master_dashboard.loc[df_master_dashboard['program'].isin(master_applications['program']), col_applications] = master_applications.loc[master_applications['program'].isin(df_master_dashboard['program']), 'applications'].values
# df_master_dashboard[col_applications] = df_master_dashboard[col_applications].fillna(0).astype(int)
# df_master_dashboard.loc[:, [col_applications]] = master_applications
# df_master_dashboard.join(master_applications, on="program", how="left", lsuffix=col_applications)

In [ ]:
df_master_dashboard[['program', col_applications, col_contracts, col_payments, col_enrollments]]

In [ ]:
df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]


In [ ]:
main_studyonline = "Общий лендинг"
try:
    df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)

    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
    df_bitrix = pd.DataFrame()

leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
# leads['program'] = insert_values()

main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

## Stage 2

In [ ]:
#%pip install jupyter-dash dash-express

In [ ]:
def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
# папки и файлы для загрузки
relative_folder = "../data/"

programs_file = "programs.xlsx"

dashboard_file = "dashboard.xlsx"

bitrix_file = "bitrix.xls"

master_file = "asav.xlsx"
master_file_sheet_name = "только онлайн"
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение" # не пусто
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении" # не пусто

bachelor_app_file = "bac_applications.xls"

bachelor_con_file = "bac_contracts.xls"

bachelor_enr_file = "bac_enrolled.xlsx"

main_studyonline = "Общий лендинг"

template_file = "template.xlsx"

# основные параметры
col_program = 'program'
col_plan = 'plan_rus'
col_leads = "Общее кол-во заявок (studyonline) ВСЕГО"
col_applications = """Кол-во регистраций в ЛК абитуриента (РФ 1 и 2 приоритет)"""
col_contracts = """Договоры (ПК) РФ"""
col_payments = """Оплаты (ПК)"""
col_enrollments = "Зачисленные (ПК)"

# дополнительные и расчетные параметры
col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
col_payments_div_plan = """Выполнение плана %"""


In [ ]:
# считывание файлов

try:
    # cчитываем базу данных програм
    print("Начинаем считывать базу программ")
    df_online_programs = pd.read_excel(relative_folder + programs_file)
    df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
    #df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
    df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    print("База программ обработана")
except:
    print("Потерялся " + programs_file + " - нужна база программ")


try:# Шаблон дашборда / template
    print("Начинаем считывать шаблон дашборда")
    # создаем дашборд по магистратурам добавляя туда программы из базы
    df_master_dashboard = pd.read_excel(relative_folder + template_file)
    df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
    df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
    df_master_dashboard = df_master_dashboard.fillna(0)
    df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)
    print("Шаблон дашборда считан")
except:
    print("Потерялся " + template_file + " - без него дашборд не собрать")


# df_bachelor_dashboard = pd.read_excel(relative_folder + template_file)

try:# Число лидов. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса")
    df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
    df_bitrix = pd.DataFrame()

leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
leads = leads * 20 #only for test! REMOVE LATER
leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
# leads[col_program] = insert_values()
df_master_dashboard[col_leads] = insert_values(df_master_dashboard, leads, 'program_bitrix', col_leads)
main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

# АСАВ
try:
    print("Начинаем считывать данные от АСАВ")
    df_master = pd.read_excel(relative_folder + master_file, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    print("Данные от АСАВ считаны")
except:
    print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
    df_master = pd.DataFrame()


In [ ]:
# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)

In [ ]:
df_master_dashboard[col_leads_total]                            = df_master_dashboard[col_leads_partners] + df_master_dashboard[col_leads]
df_master_dashboard[col_conversion_leads_to_contracts]          = df_master_dashboard[col_contracts] / df_master_dashboard[col_leads_total]
df_master_dashboard[col_needed_applications]                    = round(df_master_dashboard[col_plan]/ NEEDED_APPLICATIONS_RATIO)
df_master_dashboard[col_conversion_applications_to_contracts]   = df_master_dashboard[col_contracts] / df_master_dashboard[col_applications]
df_master_dashboard[col_conversion_contracts_to_payments]       = df_master_dashboard[col_payments]  / df_master_dashboard[col_contracts]
df_master_dashboard[col_conversion_contracts_to_enrollments]    = df_master_dashboard[col_payments]  / df_master_dashboard[col_contracts]
df_master_dashboard[col_payments_div_plan]                      = df_master_dashboard[col_payments]  / df_master_dashboard[col_plan]

In [ ]:
%pip install itables
from itables import show
show(df_master_dashboard.drop(columns=['program_bitrix']))

In [ ]:
df_main_dashboard


In [ ]:
df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
df_main_dashboard

In [ ]:
df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}


In [ ]:
df = pd.concat([df_main_dashboard, df_master_dashboard], sort=False, reset)
df

In [ ]:
df_master_dashboard.style.bar(subset=[col_payments_div_plan], caolor='#d65f5f')

In [ ]:


#df_master_dashboard.drop(columns=['program_bitrix']).to_excel(relative_folder + dashboard_file, float_format="%.2f", index=False)
# try:
#     df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file)
# except:
#     print("Нет выгрузки заявлений из АИС ПК или она называется не " + bachelor_app_file)
#     df_bachelor_app = pd.DataFrame()

# try:
#     df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file)
# except:
#     print("Нет выгрузки договоров из АИС ПК или она называется не " + bachelor_con_file)
#     df_bachelor_con = pd.DataFrame()

# try:
#     df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file)
# except:
#     print("Нет выгрузки зачислений из АИС ПК или она называется не " + bachelor_enr_file)
#     df_bachelor_enr = pd.DataFrame()

# df_master_dashboard[col_applications] =
# print(df_master.info())
# print(df_master.groupby(master_col_programs)[master_col_programs].count()) #.reset_index())

# df = pd.concat([df_master_dashboard, df_bachelor_dashboard, df_main_dashboard])



## Stage 2.5

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
# папки и файлы для загрузки
relative_folder = "../data/"

programs_file = "programs.xlsx"

dashboard_file = "dashboard.xlsx"

bitrix_file = "bitrix.xls"

master_file = "asav.xlsx"
master_file_sheet_name = "только онлайн"
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение" # не пусто
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении" # не пусто


bachelor_app_file = "bac_applications.xls"

bachelor_con_file = "bac_contracts.xls"

bachelor_enr_file = "bac_enrolled.xlsx"

bachelor_col_programs = "Конкурсная группа"
bachelor_col_programs_names = "Образовательная программа"
# bachelor_col_contracts = "Образовательная программа"
bachelor_col_payments = "Статус оплаты" # "Оплачен" или "Оплачен по квитанциям"
bachelor_col_enrollments = "Конкурс"

main_studyonline = "Общий лендинг"

template_file = "template.xlsx"

# основные параметры
col_program = 'program'
col_plan = 'plan_rus'
col_leads = "Общее кол-во заявок (studyonline) ВСЕГО"
col_applications = """Кол-во регистраций в ЛК абитуриента (РФ 1 и 2 приоритет)"""
col_contracts = """Договоры (ПК) РФ"""
col_payments = """Оплаты (ПК)"""
col_enrollments = "Зачисленные (ПК)"


# дополнительные и расчетные параметры
col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
col_payments_div_plan = """Выполнение плана %"""
col_income_1year     = "Выручка за 1 год"
col_income_all       = "Выручка за весь период обучения"
col_income_1year_hse = "Выручка за 1 год после отчислений партнерам"
col_income_all_hse   = "Выручка за весь период обучения после отчислений партнерам"

# считывание файлов

try:
    # cчитываем базу данных програм
    print("Начинаем считывать базу программ")
    df_online_programs = pd.read_excel(relative_folder + programs_file)
    df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
    #df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
    df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    print("База программ обработана")
except:
    print("!!!Потерялся " + programs_file + " - нужна база программ")
    #return "Error program database"


try:# Шаблон дашборда / template
    print("Начинаем считывать шаблон дашборда")
    # создаем дашборд по магистратурам добавляя туда программы из базы
    df_master_dashboard = pd.read_excel(relative_folder + template_file)
    df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
    df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
    df_master_dashboard = df_master_dashboard.fillna(0)
    df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)

    df_bachelor_dashboard = pd.read_excel(relative_folder + template_file)
    df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
    df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
    df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
    df_bachelor_dashboard[['plan_rus', 'plan_foreign']] = df_bachelor_dashboard[['plan_rus', 'plan_foreign']].astype(int)
    print("Шаблон дашборда считан")
except:
    print("!!!Потерялся " + template_file + " - без него дашборд не собрать")
    #return "Error dashboard template"

try:# Число лидов. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса")
    df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
    df_bitrix = pd.DataFrame()

try:
    leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
    leads = leads * 20 #only for test! REMOVE LATER
    leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
except:
    leads = pd.DataFrame(columns=['program_bitrix', 'values'])
df_master_dashboard[col_leads]   = insert_values(df_master_dashboard,   leads, 'program_bitrix', col_leads)
df_bachelor_dashboard[col_leads] = insert_values(df_bachelor_dashboard, leads, 'program_bitrix', col_leads)
main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

# АСАВ
try:
    print("Начинаем считывать данные от АСАВ")
    df_master = pd.read_excel(relative_folder + master_file, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    print("Данные от АСАВ считаны")
except:
    print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
    df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)



# АИС ПК
try:
    print("Начинаем считывать данные от АИС ПК")
    df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file, usecols="I:Z") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file, usecols="H:T") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file, usecols="E:H") #, sheet_name=master_file_sheet_name, skiprows=1)
    print("Данные от АИС ПК считаны")

    bachelor_dict = {
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации - онлайн (О К)'        ,
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации (Медиа) - онлайн (О К)',
'Компьютерные науки и анализ данных (онлайн)'                      :'Компьютерные науки и анализ данных - онлайн (О К)'      ,
'Экономический анализ (онлайн)'                                    :'Экономический анализ - онлайн (О К)'                    ,
'Дизайн  (онлайн)'                                                 :'Дизайн - онлайн (О К)'                                  ,
'Программные системы и автоматизация процессов разработки (онлайн)':'Программные системы и автоматизация процессов разработки - онлайн (О К)'
    }
except:
    print("Ошибка в обработке АИС ПК, возможно нет выгрузки из АИС ПК или она называется не:\n" + bachelor_app_file)
    print(bachelor_con_file)
    print(bachelor_enr_file)
    # df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
bachelor_applications = df_bachelor_app.groupby(bachelor_col_programs)[bachelor_col_programs].count() #.rename("program")#.sort_values(ascending=False)
bachelor_applications = pd.DataFrame({col_program:bachelor_applications.index, 'values':bachelor_applications.values})
df_bachelor_dashboard[col_applications] = insert_values(df_bachelor_dashboard, bachelor_applications, col_program, col_applications)

bachelor_contracts = df_bachelor_con.groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
bachelor_contracts = bachelor_contracts.rename(index=bachelor_dict)
bachelor_contracts = pd.DataFrame({col_program:bachelor_contracts.index, 'values':bachelor_contracts.values})
df_bachelor_dashboard[col_contracts] = insert_values(df_bachelor_dashboard, bachelor_contracts, col_program, col_contracts)

bachelor_payments = df_bachelor_con[(df_bachelor_con[bachelor_col_payments] == "Оплачен")|(df_bachelor_con[bachelor_col_payments] == "Оплачен по квитанциям")].groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
bachelor_payments = bachelor_payments.rename(index=bachelor_dict)
bachelor_payments = pd.DataFrame({col_program:bachelor_payments.index, 'values':bachelor_payments.values})
df_bachelor_dashboard[col_payments] = insert_values(df_bachelor_dashboard, bachelor_payments, col_program, col_payments)

bachelor_enrollments = df_bachelor_enr.groupby(bachelor_col_enrollments)[bachelor_col_enrollments].count()
bachelor_enrollments = pd.DataFrame({col_program:bachelor_enrollments.index, 'values':bachelor_enrollments.values})
df_bachelor_dashboard[col_enrollments] = insert_values(df_bachelor_dashboard, bachelor_enrollments, col_program, col_enrollments)


df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}
df = pd.concat([df_main_dashboard, df_master_dashboard, df_bachelor_dashboard], ignore_index=True, sort=False).drop(columns=['program_bitrix'])


# считаем второстепенные столбцы
df[col_leads_total]                            = df[col_leads_partners] + df[col_leads]
df[col_conversion_leads_to_contracts]          = df[col_contracts] / df[col_leads_total]
df[col_needed_applications]              = round(df[col_plan]/ NEEDED_APPLICATIONS_RATIO)
df[col_conversion_applications_to_contracts]   = df[col_contracts] / df[col_applications]
df[col_conversion_contracts_to_payments]       = df[col_payments]  / df[col_contracts]
df[col_conversion_contracts_to_enrollments]    = df[col_payments]  / df[col_contracts]
df[col_payments_div_plan]                      = df[col_payments]  / df[col_plan]
df[col_income_1year     ] = df['price'] * df[col_payments]
# df[col_income_all       ] = df['price'] * df[col_payments] * (2 if df['level'] == 'master' else 4)
# df[col_income_1year_hse ] = df[col_income_1year] * df['income_percent'] / 100
# df[col_income_all_hse   ] = df[col_income_all] * df['income_percent'] / 100

df.to_excel("dashboard.xlsx")

## Stage 3

In [ ]:
%pip install --upgrade google-api-python-client

In [ ]:
%pip install gspread oauth2client

In [ ]:
%cat ../credentials.json

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Define the scope and authenticate
scope = ["https://www.googleapis.com/auth/spreadsheets", "https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("../credentials.json", scope)
client = gspread.authorize(creds)

# Open Google Spreadsheet
spreadsheet = client.open("Еженедельный отчет 2025_общий")
worksheet = spreadsheet.dashboard

# Clear the worksheet before updating
worksheet.clear()

# Update the worksheet with the aggregated data
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

print("Google Spreadsheet updated successfully.")

In [ ]:
# Copyright 2018 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# [START sheets_quickstart]
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets.readonly"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "1BxiMVs0XRA5nFMdKvBdBZjgmUUqptlbs74OgvE2upms"
SAMPLE_RANGE_NAME = "Class Data!A2:E"


def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "../credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())

  try:
    service = build("sheets", "v4", credentials=creds)

    # Call the Sheets API
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values", [])

    if not values:
      print("No data found.")
      return

    print("Name, Major:")
    for row in values:
      # Print columns A and E, which correspond to indices 0 and 4.
      print(f"{row[0]}, {row[4]}")
  except HttpError as err:
    print(err)


if __name__ == "__main__":
  main()
# [END sheets_quickstart]

# Stage 4

In [ ]:
%pip install numpy pandas openpyxl lxml

In [ ]:
import pandas as pd

def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

def process_excel_files():
    import numpy as np

    # папки и файлы для загрузки
    relative_folder = "../data/"

    programs_file = "programs.xlsx"

    dashboard_file = "dashboard.xlsx"

    bitrix_file = "bitrix.xls"

    master_file = "asav.xlsx"
    master_file_sheet_name = "только онлайн"
    master_col_programs = "Конкурс в магистратуру"
    master_col_contracts = "Договор на обучение" # не пусто
    master_col_payments = "Оплата первого периода" # "Оплачено"
    master_col_enrollments = "Приказ о зачислении" # не пусто


    bachelor_app_file = "bac_applications.xls"

    bachelor_con_file = "bac_contracts.xls"

    bachelor_enr_file = "bac_enrolled.xlsx"

    bachelor_col_programs = "Конкурсная группа"
    bachelor_col_programs_names = "Образовательная программа"
    # bachelor_col_contracts = "Образовательная программа"
    bachelor_col_payments = "Статус оплаты" # "Оплачен" или "Оплачен по квитанциям"
    bachelor_col_enrollments = "Конкурс"

    main_studyonline = "Общий лендинг"

    template_file = "template.xlsx"


    # основные параметры
    col_program = 'program'
    col_plan = 'plan_rus'
    col_leads = "Общее кол-во заявок (studyonline) ВСЕГО"
    col_applications = """Кол-во регистраций в ЛК абитуриента (РФ 1 и 2 приоритет)"""
    col_contracts = """Договоры (ПК) РФ"""
    col_payments = """Оплаты (ПК)"""
    col_enrollments = "Зачисленные (ПК)"


    # дополнительные и расчетные параметры
    col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
    col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
    col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
    col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
    NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
    col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
    col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
    col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
    col_payments_div_plan = """Выполнение плана %"""
    col_income_1year     = "Выручка за 1 год"
    col_income_all       = "Выручка за весь период обучения"
    col_income_1year_hse = "Выручка за 1 год после отчислений партнерам"
    col_income_all_hse   = "Выручка за весь период обучения после отчислений партнерам"


    # считывание файлов

    try:
        # cчитываем базу данных програм
        print("Начинаем считывать базу программ")
        df_online_programs = pd.read_excel(relative_folder + programs_file)
        df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
        #df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
        df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
        df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
        print("База программ обработана")
    except:
        print("Потерялся " + programs_file + " - нужна база программ")
        return "Error program database"


    try:# Шаблон дашборда / template
        print("Начинаем считывать шаблон дашборда")
        # создаем дашборд по магистратурам добавляя туда программы из базы
        df_master_dashboard = pd.read_excel(relative_folder + template_file)
        df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
        df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
        df_master_dashboard = df_master_dashboard.fillna(0)
        df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)

        df_bachelor_dashboard = pd.read_excel(relative_folder + template_file)
        df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
        df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
        df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
        df_bachelor_dashboard[['plan_rus', 'plan_foreign']] = df_bachelor_dashboard[['plan_rus', 'plan_foreign']].astype(int)
        print("Шаблон дашборда считан")
    except:
        print("Потерялся " + template_file + " - без него дашборд не собрать")
        return "Error dashboard template"

    try:# Число лидов. Почему-то это html таблица, хотя файл xls
        print("Начинаем считывать данные от Битрикса")
        df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
        df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)
        print("Данные от Битрикса считаны")
        # pd.read_excel(relative_folder + bitrix_file)
    except:
        print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
        df_bitrix = pd.DataFrame()

    try:
        leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
        leads = leads * 20 #only for test! REMOVE LATER
        leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
    except:
        leads = pd.DataFrame(columns=['program_bitrix', 'values'])
    df_master_dashboard[col_leads] = insert_values(df_master_dashboard, leads, 'program_bitrix', col_leads)
    main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

    # АСАВ
    try:
        print("Начинаем считывать данные от АСАВ")
        df_master = pd.read_excel(relative_folder + master_file, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        print("Данные от АСАВ считаны")
    except:
        print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
        df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

    # достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
    master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
    df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

    master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
    master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
    df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

    master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
    master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
    df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

    master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
    master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
    df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)




    # АИС ПК
    try:
        print("Начинаем считывать данные от АИС ПК")
        df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file, usecols="I:Z") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file, usecols="H:T") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file, usecols="E:H") #, sheet_name=master_file_sheet_name, skiprows=1)
        print("Данные от АИС ПК считаны")

        bachelor_dict = {
    'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации - онлайн (О К)'        ,
    'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации (Медиа) - онлайн (О К)',
    'Компьютерные науки и анализ данных (онлайн)'                      :'Компьютерные науки и анализ данных - онлайн (О К)'      ,
    'Экономический анализ (онлайн)'                                    :'Экономический анализ - онлайн (О К)'                    ,
    'Дизайн  (онлайн)'                                                 :'Дизайн - онлайн (О К)'                                  ,
    'Программные системы и автоматизация процессов разработки (онлайн)':'Программные системы и автоматизация процессов разработки - онлайн (О К)'
        }
    except:
        print("Ошибка в обработке АИС ПК, возможно нет выгрузки из АИС ПК или она называется не:\n" + bachelor_app_file)
        print(bachelor_con_file)
        print(bachelor_enr_file)
        # df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

    # достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
    bachelor_applications = df_bachelor_app.groupby(bachelor_col_programs)[bachelor_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    bachelor_applications = pd.DataFrame({col_program:bachelor_applications.index, 'values':bachelor_applications.values})
    df_bachelor_dashboard[col_applications] = insert_values(df_bachelor_dashboard, bachelor_applications, col_program, col_applications)

    bachelor_contracts = df_bachelor_con.groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
    bachelor_contracts = bachelor_contracts.rename(index=bachelor_dict)
    bachelor_contracts = pd.DataFrame({col_program:bachelor_contracts.index, 'values':bachelor_contracts.values})
    df_bachelor_dashboard[col_contracts] = insert_values(df_bachelor_dashboard, bachelor_contracts, col_program, col_contracts)

    bachelor_payments = df_bachelor_con[(df_bachelor_con[bachelor_col_payments] == "Оплачен")|(df_bachelor_con[bachelor_col_payments] == "Оплачен по квитанциям")].groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
    bachelor_payments = bachelor_payments.rename(index=bachelor_dict)
    bachelor_payments = pd.DataFrame({col_program:bachelor_payments.index, 'values':bachelor_payments.values})
    df_bachelor_dashboard[col_payments] = insert_values(df_bachelor_dashboard, bachelor_payments, col_program, col_payments)

    bachelor_enrollments = df_bachelor_enr.groupby(bachelor_col_enrollments)[bachelor_col_enrollments].count()
    bachelor_enrollments = pd.DataFrame({col_program:bachelor_enrollments.index, 'values':bachelor_enrollments.values})
    df_bachelor_dashboard[col_enrollments] = insert_values(df_bachelor_dashboard, bachelor_enrollments, col_program, col_enrollments)


    df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
    df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}
    df = pd.concat([df_main_dashboard, df_master_dashboard, df_bachelor_dashboard], ignore_index=True, sort=False).drop(columns=['program_bitrix'])


    # считаем второстепенные столбцы
    df[col_leads_total]                            = df[col_leads_partners] + df[col_leads]
    df[col_conversion_leads_to_contracts]          = df[col_contracts] / df[col_leads_total]
    df[col_needed_applications]              = round(df[col_plan]/ NEEDED_APPLICATIONS_RATIO)
    df[col_conversion_applications_to_contracts]   = df[col_contracts] / df[col_applications]
    df[col_conversion_contracts_to_payments]       = df[col_payments]  / df[col_contracts]
    df[col_conversion_contracts_to_enrollments]    = df[col_payments]  / df[col_contracts]
    df[col_payments_div_plan]                      = df[col_payments]  / df[col_plan]
    df[col_income_1year     ] = df['price'] * df[col_payments]
    # df[col_income_all       ] = df['price'] * df[col_payments] * (2 if df['level'] == 'master' else 4)
    # df[col_income_1year_hse ] = df[col_income_1year] * df['income_percent'] / 100
    # df[col_income_all_hse   ] = df[col_income_all] * df['income_percent'] / 100

    df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
    df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads}
    df = pd.concat([df_main_dashboard, df_master_dashboard], ignore_index=True, sort=False) #df_bachelor_dashboard,

    df.replace(np.inf, 0, inplace=True)

    return df

In [ ]:
df = process_excel_files()

In [ ]:
display(df)

In [ ]:
from process import process_excel_files
from update import update_sheet

df = process_excel_files()


# Stage 5

In [ ]:
import pandas as pd

def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    for _, row in df_dashboard.iterrows():
        if row[col_join] in df_values[col_join]:
            row[col_values] = df_values.loc[col_join, 'values']
        else:
            row[col_values] = 0
        print(row)
    # df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    # df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]


import numpy as np

# папки и файлы для загрузки
relative_folder = "../data/"
templates_folder = "../templates/"

programs_file = "programs.xlsx"

# dashboard_file = "dashboard.xlsx"

bitrix_file = "bitrix.xls"
portal_file = "portal.xls"

master_file = "asav.xlsx"
master_file_sheet_name = "только онлайн"
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение" # не пусто
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении" # не пусто


bachelor_app_file = "bac_applications.xls"

bachelor_con_file = "bac_contracts.xls"

bachelor_enr_file = "bac_enrolled.xlsx"

bachelor_col_programs = "Конкурсная группа"
bachelor_col_programs_names = "Образовательная программа"
# bachelor_col_contracts = "Образовательная программа"
bachelor_col_payments = "Статус оплаты" # "Оплачен" или "Оплачен по квитанциям"
bachelor_col_enrollments = "Конкурс"

main_studyonline = "Общий лендинг"

template_file = "template.xlsx"


# основные параметры
col_program = 'program'
col_plan = 'plan_rus'
col_leads = "Общее кол-во заявок (studyonline) ВСЕГО"
col_applications = """Кол-во регистраций в ЛК абитуриента (РФ 1 и 2 приоритет)"""
col_contracts = """Договоры (ПК) РФ"""
col_payments = """Оплаты (ПК)"""
col_enrollments = "Зачисленные (ПК)"


# дополнительные и расчетные параметры
col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
col_payments_div_plan = """Выполнение плана %"""
col_income_1year     = "Выручка за 1 год, млн.руб."
col_income_all       = "Выручка за весь период обучения"
col_income_1year_hse = "Выручка за 1 год после отчислений партнерам"
col_income_all_hse   = "Выручка за весь период обучения после отчислений партнерам"


# считывание файлов

try:
    # cчитываем базу данных програм
    print("Начинаем считывать базу программ")
    df_online_programs = pd.read_excel(templates_folder + programs_file)
    df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
    #df_online_programs[['plan_rus', 'plan_foreign']] = df_online_programs[['plan_rus', 'plan_foreign']].astype(int)
    df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    print("База программ обработана")
except:
    print("Потерялся " + programs_file + " - нужна база программ")


try:# Шаблон дашборда / template
    print("Начинаем считывать шаблон дашборда")
    # создаем дашборд по магистратурам добавляя туда программы из базы
    df_master_dashboard = pd.read_excel(templates_folder + template_file)
    df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
    df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
    df_master_dashboard = df_master_dashboard.fillna(0)
    df_master_dashboard[['plan_rus', 'plan_foreign']] = df_master_dashboard[['plan_rus', 'plan_foreign']].astype(int)

    df_bachelor_dashboard = pd.read_excel(templates_folder + template_file)
    df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
    df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
    df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
    df_bachelor_dashboard[['plan_rus', 'plan_foreign']] = df_bachelor_dashboard[['plan_rus', 'plan_foreign']].astype(int)
    print("Шаблон дашборда считан")
except:
    print("Потерялся " + template_file + " - без него дашборд не собрать")


In [ ]:
df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]

In [ ]:

try:# Число лидов. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса")
    df_bitrix = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix['Образовательная программа'].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
    df_bitrix = pd.DataFrame()

try:# Число лидов c портала. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Портала")
    df_portal = pd.read_html(relative_folder + portal_file, header=0)[0]
    df_portal['Образовательная программа'].fillna(main_studyonline, inplace=True)
    print("Данные от Портала считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки заявок с Портала или она называется не " + portal_file)
    df_portal = pd.DataFrame()


In [ ]:
leads

In [ ]:
df_master_dashboard[col_leads]

In [ ]:
df_master_dashboard[col_leads]   = insert_values(df_master_dashboard,   leads, 'program_bitrix', col_leads)

In [ ]:

# АСАВ
try:
    print("Начинаем считывать данные от АСАВ")
    df_master = pd.read_excel(relative_folder + master_file, skiprows=1, usecols="L:DT") #sheet_name=master_file_sheet_name,
    print("Данные от АСАВ считаны")
except:
    print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
    df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])


In [ ]:
 df_master['Магистерская специализация'] = df_master['Магистерская специализация'].fillna('')


In [ ]:
df_master[~df_master['Магистерская специализация'].str.contains("офлайн")]

In [ ]:
df_master = df_master[~df_master['Магистерская специализация'].str.contains("оффлайн")]


In [ ]:

try:
    leads = df_bitrix.groupby('Образовательная программа')['Образовательная программа'].count()
    # leads = leads * 20 #only for test! REMOVE LATER
    leads = pd.DataFrame({'program_bitrix':leads.index, 'values':leads.values})
except:
    leads = pd.DataFrame(columns=['program_bitrix', 'values'])

try:
    leads_portal = df_portal.groupby('Образовательная программа')['Образовательная программа'].count()
    # leads = leads * 20 #only for test! REMOVE LATER
    leads_portal = pd.DataFrame({'program_bitrix':leads_portal.index, 'values':leads_portal.values})
except:
    leads_portal = pd.DataFrame(columns=['program_bitrix', 'values'])

df_master_dashboard[col_leads]   = insert_values(df_master_dashboard,   leads, 'program_bitrix', col_leads)
df_bachelor_dashboard[col_leads] = insert_values(df_bachelor_dashboard, leads, 'program_bitrix', col_leads)
main_leads = leads.loc[leads['program_bitrix'] == main_studyonline, 'values'].values[0]

df_master_dashboard[col_leads_partners]   = insert_values(df_master_dashboard,   leads_portal, 'program_bitrix', col_leads_partners)
df_bachelor_dashboard[col_leads_partners] = insert_values(df_bachelor_dashboard, leads_portal, 'program_bitrix', col_leads_partners)
# main_leads_portal = leads_portal.loc[leads_portal['program_bitrix'] == main_studyonline, 'values'].values[0]

# АСАВ
try:
    print("Начинаем считывать данные от АСАВ")
    df_master = pd.read_excel(relative_folder + master_file, skiprows=1, usecols="L:DT") #sheet_name=master_file_sheet_name,
    print("Данные от АСАВ считаны")
except:
    print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
    df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)


# АИС ПК
try:
    print("Начинаем считывать данные от АИС ПК")
    df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file, usecols="I:Z") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file, usecols="H:T") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file, usecols="E:H") #, sheet_name=master_file_sheet_name, skiprows=1)
    print("Данные от АИС ПК считаны")

    bachelor_dict = {
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации - онлайн (О К)'        ,
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации (Медиа) - онлайн (О К)',
'Компьютерные науки и анализ данных (онлайн)'                      :'Компьютерные науки и анализ данных - онлайн (О К)'      ,
'Экономический анализ (онлайн)'                                    :'Экономический анализ - онлайн (О К)'                    ,
'Дизайн  (онлайн)'                                                 :'Дизайн - онлайн (О К)'                                  ,
'Программные системы и автоматизация процессов разработки (онлайн)':'Программные системы и автоматизация процессов разработки - онлайн (О К)'
    }
        # достаем данные по ЛК, договорам, оплатам и зачислениям из АИС ПК
    bachelor_applications = df_bachelor_app.groupby(bachelor_col_programs)[bachelor_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    bachelor_applications = pd.DataFrame({col_program:bachelor_applications.index, 'values':bachelor_applications.values})
    df_bachelor_dashboard[col_applications] = insert_values(df_bachelor_dashboard, bachelor_applications, col_program, col_applications)

    bachelor_contracts = df_bachelor_con.groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
    bachelor_contracts = bachelor_contracts.rename(index=bachelor_dict)
    bachelor_contracts = pd.DataFrame({col_program:bachelor_contracts.index, 'values':bachelor_contracts.values})
    df_bachelor_dashboard[col_contracts] = insert_values(df_bachelor_dashboard, bachelor_contracts, col_program, col_contracts)

    bachelor_payments = df_bachelor_con[(df_bachelor_con[bachelor_col_payments] == "Оплачен")|(df_bachelor_con[bachelor_col_payments] == "Оплачен по квитанциям")].groupby(bachelor_col_programs_names)[bachelor_col_programs_names].count()
    bachelor_payments = bachelor_payments.rename(index=bachelor_dict)
    bachelor_payments = pd.DataFrame({col_program:bachelor_payments.index, 'values':bachelor_payments.values})
    df_bachelor_dashboard[col_payments] = insert_values(df_bachelor_dashboard, bachelor_payments, col_program, col_payments)

    bachelor_enrollments = df_bachelor_enr.groupby(bachelor_col_enrollments)[bachelor_col_enrollments].count()
    bachelor_enrollments = pd.DataFrame({col_program:bachelor_enrollments.index, 'values':bachelor_enrollments.values})
    df_bachelor_dashboard[col_enrollments] = insert_values(df_bachelor_dashboard, bachelor_enrollments, col_program, col_enrollments)

except:
    print("Ошибка в обработке АИС ПК, возможно нет выгрузки из АИС ПК или она называется не:\n" + bachelor_app_file)
    print(bachelor_con_file)
    print(bachelor_enr_file)
    # df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])



df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads} #, col_leads_partners: main_leads_portal}
df = pd.concat([df_main_dashboard, df_master_dashboard, df_bachelor_dashboard], ignore_index=True, sort=False).drop(columns=['program_bitrix'])


# считаем второстепенные столбцы
df[col_leads_total]                            = df[col_leads_partners] + df[col_leads]
df[col_conversion_leads_to_contracts]          = df[col_contracts] / df[col_leads_total]
df[col_needed_applications]              = round(df[col_plan]/ NEEDED_APPLICATIONS_RATIO)
df[col_conversion_applications_to_contracts]   = df[col_contracts] / df[col_applications]
df[col_conversion_contracts_to_payments]       = df[col_payments]  / df[col_contracts]
df[col_conversion_contracts_to_enrollments]    = df[col_payments]  / df[col_contracts]
df[col_payments_div_plan]                      = df[col_payments]  / df[col_plan]
df[col_income_1year     ] = df['price'] * df[col_payments] / 1000 # from thousands to millions
# df[col_income_all       ] = df['price'] * df[col_payments] * (2 if df['level'] == 'master' else 4)
# df[col_income_1year_hse ] = df[col_income_1year] * df['income_percent'] / 100
# df[col_income_all_hse   ] = df[col_income_all] * df['income_percent'] / 100

df.replace(np.inf, 0, inplace=True)
df.fillna(0, inplace=True)

display(df)

In [ ]:
%ls ..

# Stage 6

In [ ]:
import pandas as pd
import numpy as np
from col_names import *
from datetime import datetime, timedelta

templates_folder = "../templates/"
master_applications_file_2023 = "asav_2023_applications.csv"
master_contracts_file_2023 = "asav_2023_contracts.csv"
# master_applications_file_2024 = "asav_2024_applications.csv"
master_contracts_file_2024 = "asav_2024_contracts.csv"



In [ ]:
applications_dates_2023 = pd.read_csv(templates_folder + master_applications_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
contracts_dates_2023 = pd.read_csv(templates_folder + master_contracts_file_2023, parse_dates=True)
contracts_dates_2024 = pd.read_csv(templates_folder + master_contracts_file_2024, parse_dates=True)
    # applications_dates_2024 = pd.read_csv(master_applications_file_2024)

In [ ]:
applications_dates_2023.info()

In [ ]:
applications_dates_2023.sample(5)

In [ ]:

now = datetime.now()
delta_now_2023 = timedelta(days=365+366)
applications_dates_2023.where(applications_dates_2023 + delta_now_2023 < now).count()

In [ ]:
applications_2023

In [ ]:
bitrix_file_before_april = "bitrix_2024-10-01_2025-03-31.xlsx"

In [ ]:
pd.read_excel(templates_folder + bitrix_file_before_april, usecols="I:N")

In [ ]:
a = [1, 2, 3]
not a[4:]

In [ ]:
class Solution:
    memory = {}
    def mostPointsStart(self, questions: list[list[int]], start) -> int:
        if not questions[start:]:
            return 0
        else:
            # if self.memory.get(start+1) is None:
            #     self.memory[start+1] = self.mostPointsStart(questions, start + 1)
            # if self.memory.get(start + questions[start][1] + 1) is None:
            #     self.memory[start + questions[start][1] + 1] = questions[start][0] + self.mostPointsStart(questions, start + questions[start][1] + 1)
            # points_skip    = self.memory[start+1]
            # points_no_skip = questions[start][0] + self.memory[start + questions[start][1] + 1]
            # return max(points_skip, points_no_skip)

            ###
            return max(self.memory.setdefault(start + 1,
                                              self.mostPointsStart(questions, start + 1)),
                       questions[start][0]
                       + self.memory.setdefault(start + questions[start][1] + 1,
                                              self.mostPointsStart(questions, start + questions[start][1] + 1)))

            # memory
            # start  0 1
            # 1 +
    def mostPoints(self, questions: list[list[int]]) -> int:
        return self.mostPointsStart(questions, 0)

In [ ]:
questions = [[1,1],[2,2],[3,3],[4,4],[5,5]]

In [ ]:
obj = Solution()
obj.mostPoints(questions)

# Stage 7
16.04.25

In [ ]:
%pip install gspread oauth2client

In [ ]:
%ls ../data/dashboards

In [ ]:
file_name = "../data/dashboards/dashboard2025.04.13-21.48.56.xlsx"
(pd.read_excel(file_name)[[col_leads, col_applications]]).to_csv("../templates/prev_data.csv")

In [ ]:
import numpy as np
import gspread
from gspread.utils import ValueRenderOption
import pandas as pd
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials
from col_names import *

def update_sheet(aggregated_data, update_delta=False, history_data=None):

    prev_file = "templates/prev_data.csv"
    # define the scope
    scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']

    # add credentials to the account
    creds = ServiceAccountCredentials.from_json_keyfile_name('/Users/iasonov/Programming/Python/HSE_dashboard/service_credentials.json', scope)

    # authorize the clientsheet
    client = gspread.authorize(creds)

    # get the instance of the Spreadsheet
    sheet = client.open('Еженедельный отчет 2025_общий')

    # get the first sheet of the Spreadsheet
    dashboard_sales = sheet.get_worksheet(0)

    print("Гугл-дашборд открыт")

    str_time = datetime.now().strftime("%H:%M")
    str_date = datetime.now().strftime("%d.%m")

    if update_delta:
        # TODO - replace using formulae (not absolute range)
        try:
            df_prev = pd.read_csv(prev_file)
            prev_leads = df_prev[col_leads]
            prev_applications = df_prev[col_applications]
            print("Данные о лидах с прошлого обновления считаны")
        except:
            prev_leads        = np.array(dashboard_sales.get("J2:J42", value_render_option=ValueRenderOption.unformatted))[:,0]
            prev_applications = np.array(dashboard_sales.get("O2:O42", value_render_option=ValueRenderOption.unformatted))[:,0]
            print("Данные в " + prev_file + " на локальном диске не найдены, считаю дельту относительно гугл-дашборда")
        aggregated_data[col_leads_delta]        = aggregated_data[col_leads]        - prev_leads
        aggregated_data[col_applications_delta] = aggregated_data[col_applications] - prev_applications
        aggregated_data[[col_leads, col_applications]].to_csv(prev_file)
        print("Сведения о лидах и регистрациях в ЛК за полнедели обновлены")
    else:
        prev_leads_delta        = np.array(dashboard_sales.get("L2:L42", value_render_option=ValueRenderOption.unformatted))
        prev_applications_delta = np.array(dashboard_sales.get("P2:P42", value_render_option=ValueRenderOption.unformatted))
        aggregated_data[col_leads_delta]        = prev_leads_delta[:,0]
        aggregated_data[col_applications_delta] = prev_applications_delta[:,0]
        print("Данные за полнедели не обновляются")

    # dashboard_sales.update([aggregated_data.columns.values.tolist()] + aggregated_data.values.tolist())
    print("Данные в гугл-дашборд записаны")

    # TODO replace without absolute cell indexes
    # dashboard_sales.update_acell('B43', str_time + ", " + str_date + ".2025") # 2025
    if history_data is not None:
        print("добавляем данные за 23 и 24 гг.")

    return

In [ ]:

update_deltas = True # each monday and thursday
update_google_dashboard = True

if update_google_dashboard:
  update_sheet(pd.read_excel(file_name), update_deltas, None)

# Stage 8
22.04.25

In [ ]:
%pip install html5lib bs4 xlrd

In [ ]:
# df_bitrix = pd.read_excel('../data/bitrix.xls', encoding='latin1')
df_bitrix_after_april = pd.read_excel('../data/bitrix.xlsx', header=0)

In [ ]:
df_bitrix_after_april['Образовательная программа']

В 
n
n кластерах расположены серверы, на которых установлена одинаковая операционная система. В её ядре обнаружена уязвимость, поэтому необходимо обновить серверы до новой версии ядра.

В 
i
i-м кластере расположено 
x
i
x 
i
​
  серверов. Обновление одного сервера занимает одну единицу времени. Обновление серверов происходит последовательно, поэтому общее время обновления всех серверов в 
i
i-м кластере составляет 
x
i
x 
i
​
  единиц времени.

Процесс обновления в одном кластере нельзя прерывать до тех пор, пока все серверы в нём не будут обновлены. Также нельзя одновременно обновлять серверы в двух кластерах.

Для каждого кластера выделено окно времени 
[
a
i
,
a
i
+
x
i
]
[a 
i
​
 ,a 
i
​
 +x 
i
​
 ], в течение которого можно проводить обновление. Эти окна могут пересекаться.

Необходимо выбрать кластеры, в которых будет проведено обновление, чтобы на как можно большем количестве серверов была установлена новая версия ядра.

Формат ввода
В первой строке дано число кластеров 
n
n 
(
1
≤
n
≤
1
0
5
)
(1≤n≤10 
5
 ).

В каждой из 
n
n следующих строк дана пара чисел 
a
i
a 
i
​
  и 
x
i
x 
i
​
 , разделённая пробелом 
(
1
≤
a
i
,
x
i
≤
1
0
9
)
(1≤a 
i
​
 ,x 
i
​
 ≤10 
9
 ).

Формат вывода
В первой строке выходного файла нужно вывести максимальное суммарное количество серверов, на которых возможно обновить ядро.

Во второй строке выходного файла нужно вывести разделённые пробелом номера кластеров, в которых следует обновить ядро. Кластеры нумеруются с нуля. Номера можно вывести в произвольном порядке.

In [ ]:
reader = open('input.txt', 'r')
a, b = [int(n) for n in reader.readline().split(" ")]
reader.close()

writer = open('output.txt', 'w')
writer.write("%d" % (a+b))
writer.close()

# Stage 9
27.04.25

In [ ]:
master_col_programs = "Конкурс в магистратуру"
master_col_contracts = "Договор на обучение" # не пусто
master_col_payments = "Оплата первого периода" # "Оплачено"
master_col_enrollments = "Приказ о зачислении" # не пусто

bachelor_col_programs = "Конкурсная группа"
# bachelor_col_contracts = "Образовательная программа"
bachelor_col_payments = "Статус оплаты" # "Оплачен" или "Оплачен по квитанциям"
bachelor_col_enrollments = "Конкурс"


# основные параметры
col_program = 'program'
col_programs_names = "Образовательная программа"
col_plan_rus = 'plan_rus'
col_plan_foreign = 'plan_foreign'
col_leads = "Кол-во заявок (studyonline) с 1.10"
col_leads_after_april = """Кол-во заявок (studyonline) с 1.04"""
col_leads_after_april_prev = """Прошлогоднее Кол-во заявок (studyonline) с 1.04"""
col_applications = """Регистрации в ЛК (РФ все приоритеты)"""
col_applications_prev = """Прошлогодние Регистрации в ЛК (РФ все приоритеты)"""
col_contracts = """Договоры (ПК) РФ"""
col_contracts_prev =  """Прошлогодние Договоры (ПК) РФ"""
col_payments = """Оплаты (ПК)"""
col_enrollments = "Зачисленные (ПК)"
col_gender_asav = "Пол"
col_birthday = "Дата рождения"
col_male = "male"
col_female	= "female"
col_ages = "ages"
col_ages_mean = "ages_mean"


# дополнительные и расчетные параметры
col_leads_total = """Общее кол-во заявок ВСЕГО (портал+РК)"""
col_leads_partners = """Общее кол-во заявок  c  hse.ru по кнопкам/партнерских стр"""
col_conversion_leads_to_contracts = """Конверсия заявка -> договор (без учета заявок с общего ленда)"""
col_needed_applications = """Необходимо регистраций в ЛК для обеспечения набора (30% поступили от регистрации в АСАВ)"""
col_conversion_applications_to_contracts = """Конверсия ЛК -> договор"""
col_conversion_contracts_to_payments = """Конверсия договор -> оплата"""
col_conversion_contracts_to_enrollments = """Конверсия договор -> зачисление"""
col_payments_div_plan = """Выполнение плана %"""
col_income_1year     = "Выручка за 1 год, млн.руб."
col_income_all       = "Выручка за весь период обучения"
col_income_1year_hse = "Выручка за 1 год после отчислений партнерам"
col_income_all_hse   = "Выручка за весь период обучения после отчислений партнерам"
col_leads_delta = """Общее кол-во заявок (studyonline) за 1\\2 недели"""
col_applications_delta = """Кол-во регистраций в ЛК абитуриента за 1\\2 недели"""

#asav
col_id_asav = "Рег. номер"

#dashboard unused
col_gender = "gender"
col_ages_bars = "ages_bars"



In [ ]:
import pandas as pd
import numpy as np
from col_names import *
from datetime import datetime, timedelta

def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    for i, row in df_dashboard.iterrows():
        if row[col_join] in df_values[col_join].values:
            df_dashboard.loc[i, col_values] = df_values[df_values[col_join] == row[col_join]].values[0][1] # row[col_values]
            #row[col_values] = df_values[(row[col_join], 'values')]
        else:
            df_dashboard.loc[i, col_values] = 0
        # print(row)
    # df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    # df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents
main_studyonline = "Общий лендинг"

# папки и файлы для загрузки
relative_folder = "../data/"
templates_folder = "../templates/"

programs_file = "programs.xlsx"
template_file = "template.xlsx"

# dashboard_file = "dashboard.xlsx"

bitrix_file = "bitrix.xls"
bitrix_file_before_april = "bitrix_2024-10-01_2025-03-31.xlsx"
portal_file = "portal.xls"

master_file = "asav.xlsx"
# master_file_sheet_name = "только онлайн"

bachelor_app_file = "bac_applications.xls"
bachelor_con_file = "bac_contracts.xls"
bachelor_enr_file = "bac_enrolled.xlsx"

# считывание файлов
try:
    # cчитываем базу данных програм
    print("Начинаем считывать базу программ")
    df_online_programs = pd.read_excel(templates_folder + programs_file)
    df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
    df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    print("База программ обработана")
except:
    print("Потерялся " + programs_file + " - нужна база программ")


try:# Шаблон дашборда / template
    print("Начинаем считывать шаблон дашборда")
    # создаем дашборд по магистратурам добавляя туда программы из базы
    df_master_dashboard = pd.read_excel(templates_folder + template_file)
    df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
    df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
    df_master_dashboard = df_master_dashboard.fillna(0)
    df_master_dashboard[[col_plan_rus, col_plan_foreign]] = df_master_dashboard[[col_plan_rus, col_plan_foreign]].astype(int)

    df_bachelor_dashboard = pd.read_excel(templates_folder + template_file)
    df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
    df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
    df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
    df_bachelor_dashboard[[col_plan_rus, col_plan_foreign]] = df_bachelor_dashboard[[col_plan_rus, col_plan_foreign]].astype(int)
    print("Шаблон дашборда считан")
except:
    print("Потерялся " + template_file + " - без него дашборд не собрать")

try:# Число лидов со studyonline с 1 апреля по настоящее время. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса в html-формате")
    df_bitrix_after_april = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix_after_april[col_programs_names].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    try:# Число лидов со studyonline с 1 апреля по настоящее время. На случай, если html чтение не сработало
        print("Начинаем считывать данные от Битрикса в xls-формате")
        df_bitrix_after_april = pd.read_excel(relative_folder + bitrix_file, header=0)
        df_bitrix_after_april[col_programs_names].fillna(main_studyonline, inplace=True)
        print("Данные от Битрикса считаны")
        # pd.read_excel(relative_folder + bitrix_file)
    except:
        print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
        df_bitrix_after_april = pd.DataFrame()

try:# Число лидов c портала c 1 октября по настоящее время. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Портала")
    df_portal = pd.read_html(relative_folder + portal_file, header=0)[0]
    df_portal[col_programs_names].fillna(main_studyonline, inplace=True)
    print("Данные от Портала считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки заявок с Портала или она называется не " + portal_file)
    df_portal = pd.DataFrame()

try:# Число лидов из битрикс до 1 апреля (не включительно). Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса до 31.03")
    df_bitrix_before_april = pd.read_excel(templates_folder + bitrix_file_before_april, usecols="I:N")
    df_bitrix_before_april[col_programs_names].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса до 31.03 считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    print("Нет выгрузки заявок из Битрикса до 31.03 или она называется не " + bitrix_file_before_april)
    df_bitrix_before_april = pd.DataFrame()

try:
    leads_after_april = df_bitrix_after_april.groupby(col_programs_names)[col_programs_names].count()
    leads_after_april = pd.DataFrame({'program_bitrix':leads_after_april.index, 'values':leads_after_april.values})
except:
    leads_after_april = pd.DataFrame(columns=['program_bitrix', 'values'])

try:
    leads_before_april = df_bitrix_before_april.groupby(col_programs_names)[col_programs_names].count()
    leads_before_april = pd.DataFrame({'program_bitrix':leads_before_april.index, 'values':leads_before_april.values})
except:
    leads_before_april = pd.DataFrame(columns=['program_bitrix', 'values'])

#     # достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
# master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
# master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
# df_master_dashboard[col_applications_since_april] = insert_values(df_master_dashboard, master_applications, col_program, col_applications_since_april)


# try:
#     df_master_2024 = pd.read_csv(templates_folder + master_file_before_april, usecols="J:T")
#     master_applications_2024 = df_master_2024.groupby(master_col_programs)[master_col_programs].count()
#     master_applications_2024 = pd.DataFrame({col_program:master_applications_2024.index, 'values':master_applications_2024.values})
#     df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications_2024, col_program, col_applications)
# except:
#     print("Ошибка при чтении файла asav_2024-10-01_2025-03-31.csv")
#     df_master_dashboard[col_applications] = 0


try:
    leads_portal = df_portal.groupby(col_programs_names)[col_programs_names].count()
    leads_portal = pd.DataFrame({'program_bitrix':leads_portal.index, 'values':leads_portal.values})
except:
    leads_portal = pd.DataFrame(columns=['program_bitrix', 'values'])


df_master_dashboard  [col_leads] = insert_values(df_master_dashboard,   leads_before_april, 'program_bitrix', col_leads)
df_bachelor_dashboard[col_leads] = insert_values(df_bachelor_dashboard, leads_before_april, 'program_bitrix', col_leads)

df_master_dashboard  [col_leads_after_april] = insert_values(df_master_dashboard,   leads_after_april, 'program_bitrix', col_leads_after_april)
df_bachelor_dashboard[col_leads_after_april] = insert_values(df_bachelor_dashboard, leads_after_april, 'program_bitrix', col_leads_after_april)

df_master_dashboard  [col_leads] += df_master_dashboard  [col_leads_after_april] # TODO check
df_bachelor_dashboard[col_leads] += df_bachelor_dashboard[col_leads_after_april]

main_leads             = leads_before_april.loc[leads_before_april['program_bitrix'] == main_studyonline, 'values'].values[0]
main_leads_after_april = leads_after_april.loc[leads_after_april['program_bitrix'] == main_studyonline, 'values'].values[0]
main_leads            += main_leads_after_april # TODO check

df_master_dashboard  [col_leads_partners] = insert_values(df_master_dashboard,   leads_portal, 'program_bitrix', col_leads_partners)
df_bachelor_dashboard[col_leads_partners] = insert_values(df_bachelor_dashboard, leads_portal, 'program_bitrix', col_leads_partners)
# main_leads_portal = leads_portal.loc[leads_portal['program_bitrix'] == main_studyonline, 'values'].values[0]

# АСАВ
try:
    print("Начинаем считывать данные от АСАВ")
    df_master = pd.read_excel(relative_folder + master_file, skiprows=1, usecols="E:DT") #sheet_name=master_file_sheet_name,
    print("Данные от АСАВ считаны")
except:
    print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
    df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

# убираем офлайн-психов TODO - проверить международный бизнес и другие программы с треками
df_master['Магистерская специализация'] = df_master['Магистерская специализация'].fillna('')
df_master = df_master[~df_master['Магистерская специализация'].str.contains("офлайн")]

# достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)

master_male = df_master[df_master[col_gender] == "Муж."].groupby(master_col_programs)[master_col_programs].count()
master_male = pd.DataFrame({col_program:master_male.index, 'values':master_male.values})
df_master_dashboard[col_male_asav] = insert_values(df_master_dashboard, master_male, col_program, col_male_asav)

master_female = df_master[df_master[col_gender] == "Жен."].groupby(master_col_programs)[master_col_programs].count()
master_female = pd.DataFrame({col_program:master_female.index, 'values':master_female.values})
df_master_dashboard[col_female_asav] = insert_values(df_master_dashboard, master_female, col_program, col_female_asav)



In [ ]:
df_master = df_master.dropna(subset=[col_birthday])

In [ ]:
df_master.info()

In [ ]:
df_master[col_birthday] = pd.to_datetime(df_master[col_birthday]).apply(num_years)
df_master[col_birthday]

In [ ]:
df_master.groupby(master_col_programs)[col_birthday].mean()

In [ ]:
master_birthday = pd.to_datetime(df_master[col_birthday]).apply(num_years)
print(master_birthday)

In [ ]:
def categorize_ages(age_column):
    # Определяем диапазоны
    bins = [0, 17, 23, 29, 35, 41, 47, float('inf')]
    labels = ['0-17', '18-23', '24-29', '30-35', '36-41', '42-47', '48+']

    # Используем pd.cut для разбиения на интервалы
    categories = pd.cut(age_column, bins=bins, labels=labels, right=True, include_lowest=True)

    # Считаем количество в каждом диапазоне
    counts = categories.value_counts().sort_index()

    return np.array2string(counts.values, separator=";")[1:-1]

categorize_ages(master_birthday)




In [ ]:
def yearsago(years, from_date=None):
    if from_date is None:
        from_date = datetime.now()
    try:
        return from_date.replace(year=from_date.year - years)
    except ValueError:
        # Must be 2/29!
        assert from_date.month == 2 and from_date.day == 29 # can be removed
        return from_date.replace(month=2, day=28,
                                 year=from_date.year-years)

def num_years(begin, end=None):
    if end is None:
        end = datetime.now()
    num_years = int((end - begin).days / 365.2425)
    if begin > yearsago(num_years, end):
        return num_years - 1
    else:
        return num_years

In [ ]:
str

In [ ]:
df_master

In [ ]:
def cells_from_row(row):
  return "I" + str(row.name+2) + ":J" + str(row.name+2)
df_master.apply(cells_from_row, axis=1)

In [ ]:
from datetime import datetime
import pandas as pd

str = datetime.now().strftime("%Y.%m.%d-%H.%M.%S")

In [ ]:

master_birthday = pd.to_datetime(df_master[col_birthday])
print(master_birthday)


# АИС ПК
try:
    print("Начинаем считывать данные от АИС ПК")
    df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file, usecols="I:Z") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file, usecols="H:T") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
    df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file, usecols="E:H") #, sheet_name=master_file_sheet_name, skiprows=1)
    print("Данные от АИС ПК считаны")

    bachelor_dict = {
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации - онлайн (О К)'        ,
'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации (Медиа) - онлайн (О К)',
'Компьютерные науки и анализ данных (онлайн)'                      :'Компьютерные науки и анализ данных - онлайн (О К)'      ,
'Экономический анализ (онлайн)'                                    :'Экономический анализ - онлайн (О К)'                    ,
'Дизайн  (онлайн)'                                                 :'Дизайн - онлайн (О К)'                                  ,
'Программные системы и автоматизация процессов разработки (онлайн)':'Программные системы и автоматизация процессов разработки - онлайн (О К)'
    }
        # достаем данные по ЛК, договорам, оплатам и зачислениям из АИС ПК
    bachelor_applications = df_bachelor_app.groupby(bachelor_col_programs)[bachelor_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    bachelor_applications = pd.DataFrame({col_program:bachelor_applications.index, 'values':bachelor_applications.values})
    df_bachelor_dashboard[col_applications] = insert_values(df_bachelor_dashboard, bachelor_applications, col_program, col_applications)

    bachelor_contracts = df_bachelor_con.groupby(col_programs_names)[col_programs_names].count()
    bachelor_contracts = bachelor_contracts.rename(index=bachelor_dict)
    bachelor_contracts = pd.DataFrame({col_program:bachelor_contracts.index, 'values':bachelor_contracts.values})
    df_bachelor_dashboard[col_contracts] = insert_values(df_bachelor_dashboard, bachelor_contracts, col_program, col_contracts)

    bachelor_payments = df_bachelor_con[(df_bachelor_con[bachelor_col_payments] == "Оплачен")|(df_bachelor_con[bachelor_col_payments] == "Оплачен по квитанциям")].groupby(col_programs_names)[col_programs_names].count()
    bachelor_payments = bachelor_payments.rename(index=bachelor_dict)
    bachelor_payments = pd.DataFrame({col_program:bachelor_payments.index, 'values':bachelor_payments.values})
    df_bachelor_dashboard[col_payments] = insert_values(df_bachelor_dashboard, bachelor_payments, col_program, col_payments)

    bachelor_enrollments = df_bachelor_enr.groupby(bachelor_col_enrollments)[bachelor_col_enrollments].count()
    bachelor_enrollments = pd.DataFrame({col_program:bachelor_enrollments.index, 'values':bachelor_enrollments.values})
    df_bachelor_dashboard[col_enrollments] = insert_values(df_bachelor_dashboard, bachelor_enrollments, col_program, col_enrollments)

except:
    print("Ошибка в обработке АИС ПК, возможно нет выгрузки из АИС ПК или она называется не:\n" + bachelor_app_file)
    print(bachelor_con_file)
    print(bachelor_enr_file)
    # df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])



df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_leads: main_leads, col_leads_after_april: main_leads_after_april}
df = pd.concat([df_main_dashboard, df_master_dashboard, df_bachelor_dashboard], ignore_index=True, sort=False).drop(columns=['program_bitrix'])

df.fillna(0, inplace=True)

# считаем второстепенные столбцы
df[col_leads_total]                            = df[col_leads_partners] + df[col_leads]
df[col_conversion_leads_to_contracts]          = df[col_contracts] / df[col_leads_total]
df[col_needed_applications]              = round(df[col_plan_rus]/ NEEDED_APPLICATIONS_RATIO)
df[col_conversion_applications_to_contracts]   = df[col_contracts] / df[col_applications]
df[col_conversion_contracts_to_payments]       = df[col_payments]  / df[col_contracts]
df[col_conversion_contracts_to_enrollments]    = df[col_payments]  / df[col_contracts]
df[col_payments_div_plan]                      = df[col_payments]  / df[col_plan_rus]
df[col_income_1year     ] = df['price'] * df[col_payments] / 1000 # from thousands to millions
df.loc[df['level'] == 'master', col_income_all]   = df[col_income_1year] * 2
df.loc[df['level'] == 'bachelor', col_income_all] = df[col_income_1year] * 4
# df[col_income_all       ] = df[col_income_1year]  * (2 if df['level'] == 'master' else 4) # TODO check later
df[col_income_1year_hse ] = df[col_income_1year] * df['income_percent'] / 100
df[col_income_all_hse   ] = df[col_income_all] * df['income_percent'] / 100

df.replace(np.inf, 0, inplace=True)
df.fillna(0, inplace=True)

In [ ]:
asav_2023 = pd.read_excel(templates_folder + "asav_2023.xlsx", parse_dates=[0, 1], skiprows=1, date_format="%d.%m.%Y")
asav_2023['applications_dates'] = pd.to_datetime(asav_2023['applications_dates'], format='%Y-%m-%d 00:00:00')
asav_2023['contracts_dates'] = pd.to_datetime(asav_2023['contracts_dates'], errors='coerce', format='%d.%m.%Y')
asav_2023.head()

In [ ]:
asav_2023.info()

In [ ]:
asav_2023.head()

In [ ]:
asav_2023[asav_2023['contracts_dates'] < datetime(2023, 4, 28)]

In [ ]:
templates_folder = "../templates/"
master_leads_file_2023        = "bitrix_2023-04-01_2023-09-15.csv"
master_leads_file_2024        = "bitrix_2024-04-01_2024-09-15.csv"
master_applications_file_2023 = "asav_2023_applications.csv"
master_contracts_file_2023    = "asav_2023_contracts.csv"
master_applications_file_2024 = "asav_2024_applications.csv"
master_contracts_file_2024    = "asav_2024_contracts.csv"
asav_file_2023                = "asav_2023.xlsx"
asav_file_2024                = "asav_2024.xlsx"

# TODO вписать сводные расчеты

try:
    applications_dates_2023 = pd.read_csv(templates_folder + master_applications_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
    applications_dates_2024 = pd.read_csv(templates_folder + master_applications_file_2024, parse_dates=[0], date_format="%d.%m.%Y")
    contracts_dates_2023 = pd.read_csv(templates_folder + master_contracts_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
    contracts_dates_2024 = pd.read_csv(templates_folder + master_contracts_file_2024, parse_dates=[0], date_format="%d.%m.%Y")
    leads_dates_2023 = pd.read_csv(templates_folder + master_leads_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
    leads_dates_2024 = pd.read_csv(templates_folder + master_leads_file_2024, parse_dates=[0], date_format="%d.%m.%Y")

    asav_2023 = pd.read_excel(templates_folder + asav_file_2023, parse_dates=[0, 1], skiprows=1, date_format="%d.%m.%Y")
    asav_2023['applications_dates'] = pd.to_datetime(asav_2023['applications_dates'], format='%Y-%m-%d 00:00:00')
    asav_2023['contracts_dates'] = pd.to_datetime(asav_2023['contracts_dates'], errors='coerce', format='%d.%m.%Y')

    asav_2024 = pd.read_excel(templates_folder + asav_file_2024, parse_dates=[0, 1], skiprows=1, date_format="%d.%m.%Y")
    asav_2024['applications_dates'] = pd.to_datetime(asav_2024['applications_dates'], format='%Y-%m-%d 00:00:00')
    asav_2024['contracts_dates'] = pd.to_datetime(asav_2024['contracts_dates'], errors='coerce', format='%d.%m.%Y')

except:
    print("Files of previous years are not founded or have errors", master_applications_file_2023)
    print(master_contracts_file_2023)
    print(master_applications_file_2024)
    print(master_contracts_file_2024)
    print(master_leads_file_2023)
    print(master_leads_file_2024)

now = datetime.now()
delta_now_2023 = timedelta(days=365+366) # TODO check both deltas
delta_now_2024 = timedelta(days=366)

df_pivot = pd.DataFrame.from_dict({'leads' :
                              {2023: leads_dates_2023.where(leads_dates_2023 + delta_now_2023 <= now).count(),
                               2024: leads_dates_2024.where(leads_dates_2024 + delta_now_2024 <= now).count()},
                              # 'applications_old' :
                              # {2023: applications_dates_2023.where(applications_dates_2023 + delta_now_2023 <= now).count(),
                              #  2024: applications_dates_2024.where(applications_dates_2024 + delta_now_2024 <= now).count()},
                              # 'contracts_old' :
                              # {2023: contracts_dates_2023.where(contracts_dates_2023 + delta_now_2023 <= now).count(),
                              #  2024: contracts_dates_2024.where(contracts_dates_2024 + delta_now_2024 <= now).count()},
                              'applications' :
                              {2023: asav_2023[asav_2023['applications_dates'] + delta_now_2023 <= now]['applications_dates'].count(),
                               2024: asav_2024[asav_2024['applications_dates'] + delta_now_2024 <= now]['applications_dates'].count()},
                              'contracts' :
                              {2023: asav_2023[asav_2023['contracts_dates'] + delta_now_2023 <= now]['contracts_dates'].count(),
                               2024: asav_2024[asav_2024['contracts_dates'] + delta_now_2024 <= now]['contracts_dates'].count()},
                            }) #columns=['year', 'applications', 'contracts']
df_pivot

In [ ]:

def process_history_files():
    templates_folder = "../templates/"
    master_leads_file_2023        = "bitrix_2023-04-01_2023-09-15.csv"
    master_leads_file_2024        = "bitrix_2024-04-01_2024-09-15.csv"
    # master_applications_file_2023 = "asav_2023_applications.csv"
    # master_contracts_file_2023    = "asav_2023_contracts.csv"
    # master_applications_file_2024 = "asav_2024_applications.csv"
    # master_contracts_file_2024    = "asav_2024_contracts.csv"
    asav_file_2023                = "asav_2023.xlsx"
    asav_file_2024                = "asav_2024.xlsx"

    # TODO вписать сводные расчеты

    try:
        # applications_dates_2023 = pd.read_csv(templates_folder + master_applications_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
        # applications_dates_2024 = pd.read_csv(templates_folder + master_applications_file_2024, parse_dates=[0], date_format="%d.%m.%Y")
        # contracts_dates_2023 = pd.read_csv(templates_folder + master_contracts_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
        # contracts_dates_2024 = pd.read_csv(templates_folder + master_contracts_file_2024, parse_dates=[0], date_format="%d.%m.%Y")
        leads_dates_2023 = pd.read_csv(templates_folder + master_leads_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
        leads_dates_2024 = pd.read_csv(templates_folder + master_leads_file_2024, parse_dates=[0], date_format="%d.%m.%Y")

        asav_2023 = pd.read_excel(templates_folder + asav_file_2023, parse_dates=[0, 1], skiprows=1, date_format="%d.%m.%Y")
        asav_2023['applications_dates'] = pd.to_datetime(asav_2023['applications_dates'], format='%Y-%m-%d 00:00:00')
        asav_2023['contracts_dates'] = pd.to_datetime(asav_2023['contracts_dates'], errors='coerce', format='%d.%m.%Y')

        asav_2024 = pd.read_excel(templates_folder + asav_file_2024, parse_dates=[0, 1], skiprows=1, date_format="%d.%m.%Y")
        asav_2024['applications_dates'] = pd.to_datetime(asav_2024['applications_dates'], format='%Y-%m-%d 00:00:00')
        asav_2024['contracts_dates'] = pd.to_datetime(asav_2024['contracts_dates'], errors='coerce', format='%d.%m.%Y')

    except:
        print("Files of previous years are not founded or have errors")
        # , master_applications_file_2023)
        # print(master_contracts_file_2023)
        # print(master_applications_file_2024)
        # print(master_contracts_file_2024)
        print(master_leads_file_2023)
        print(master_leads_file_2024)

    now = datetime.now()
    delta_now_2023 = timedelta(days=365+366) # TODO check both deltas
    delta_now_2024 = timedelta(days=366)

    asav_2023_no_duplicates = asav_2023.drop_duplicates(subset=[col_id_asav])
    asav_2024_no_duplicates = asav_2024.drop_duplicates(subset=[col_id_asav])


    df_pivot = pd.DataFrame.from_dict({'leads' :
                                {2023: leads_dates_2023.where(leads_dates_2023 + delta_now_2023 <= now).count(),
                                 2024: leads_dates_2024.where(leads_dates_2024 + delta_now_2024 <= now).count()},
                                # 'applications_old' :
                                # {2023: applications_dates_2023.where(applications_dates_2023 + delta_now_2023 <= now).count(),
                                #  2024: applications_dates_2024.where(applications_dates_2024 + delta_now_2024 <= now).count()},
                                # 'contracts_old' :
                                # {2023: contracts_dates_2023.where(contracts_dates_2023 + delta_now_2023 <= now).count(),
                                #  2024: contracts_dates_2024.where(contracts_dates_2024 + delta_now_2024 <= now).count()},
                                'applications' :
                                {2023: asav_2023[asav_2023['applications_dates'] + delta_now_2023 <= now]['applications_dates'].count(),
                                 2024: asav_2024[asav_2024['applications_dates'] + delta_now_2024 <= now]['applications_dates'].count()},
                                'contracts' :
                                {2023: asav_2023[asav_2023['contracts_dates'] + delta_now_2023 <= now]['contracts_dates'].count(),
                                 2024: asav_2024[asav_2024['contracts_dates'] + delta_now_2024 <= now]['contracts_dates'].count()},
                                'applications_unique' :
                                {2023: asav_2023_no_duplicates[asav_2023_no_duplicates['applications_dates'] + delta_now_2023 <= now]['applications_dates'].count(),
                                 2024: asav_2024_no_duplicates[asav_2024_no_duplicates['applications_dates'] + delta_now_2024 <= now]['applications_dates'].count()}
                                }) #columns=['year', 'applications', 'contracts']
    df_applications_prev = asav_2024[asav_2024['applications_dates'] + delta_now_2024 <= now].groupby(master_col_programs)[master_col_programs].count()
    df_contracts_prev    = asav_2024[asav_2024['contracts_dates']    + delta_now_2024 <= now].groupby(master_col_programs)[master_col_programs].count()
    return df_pivot, df_applications_prev, df_contracts_prev


In [ ]:
df_history, df_applications_prev, df_contracts_prev = process_history_files() # Only for master for now


In [ ]:
df_contracts_prev

# Stage 10
15.06.2025

In [ ]:
import pandas as pd
import numpy as np
from col_names import *
from datetime import datetime, timedelta

In [ ]:


def categorize_ages(age_column):
    # Определяем диапазоны
    bins = [0, 17, 23, 29, 35, 41, 47, float('inf')]
    labels = ['0-17', '18-23', '24-29', '30-35', '36-41', '42-47', '48+']

    # Используем pd.cut для разбиения на интервалы
    categories = pd.cut(age_column, bins=bins, labels=labels, right=True, include_lowest=True)

    # Считаем количество в каждом диапазоне
    counts = categories.value_counts().sort_index()

    return np.array2string(counts.values, separator=";")[1:-1]

def years_ago(years, from_date=None):
    if from_date is None:
        from_date = datetime.now()
    try:
        return from_date.replace(year=from_date.year - years)
    except ValueError:
        # Must be 2/29!
        assert from_date.month == 2 and from_date.day == 29 # can be removed
        return from_date.replace(month=2, day=28,
                                 year=from_date.year-years)

def num_years(begin, end=None):
    if end is None:
        end = datetime.now()
    num_years = int((end - begin).days / 365.2425)
    if begin > years_ago(num_years, end):
        return num_years - 1
    else:
        return num_years

def insert_values(df_dashboard, df_values, col_join, col_values): # df_values should have "values" column
    for i, row in df_dashboard.iterrows():
        if row[col_join] in df_values[col_join].values:
            df_dashboard.loc[i, col_values] = df_values[df_values[col_join] == row[col_join]].values[0][1] # row[col_values]
            #row[col_values] = df_values[(row[col_join], 'values')]
        else:
            df_dashboard.loc[i, col_values] = 0
        # print(row)
    # df_dashboard.loc[df_dashboard[col_join].isin(df_values[col_join]), col_values] = df_values.loc[df_values[col_join].isin(df_dashboard[col_join]), 'values'].values
    # df_dashboard[col_values] = df_dashboard[col_values].fillna(0).astype(int)
    return df_dashboard[col_values]

def process_history_files():
    templates_folder = "templates/"
    master_leads_file_2023        = "bitrix_2023-04-01_2023-09-15.csv"
    master_leads_file_2024        = "bitrix_2024-04-01_2024-09-15.csv"
    bitrix_file_2024              = "bitrix_2024-04-01_2024-09-15.xlsx"
    # master_applications_file_2023 = "asav_2023_applications.csv"
    # master_contracts_file_2023    = "asav_2023_contracts.csv"
    # master_applications_file_2024 = "asav_2024_applications.csv"
    # master_contracts_file_2024    = "asav_2024_contracts.csv"
    asav_file_2023                = "asav_2023.xlsx"
    asav_file_2024                = "asav_2024.xlsx"

    # TODO вписать сводные расчеты

    print("Начинаем считывать исторические данные")

    try:
        # applications_dates_2023 = pd.read_csv(templates_folder + master_applications_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
        # applications_dates_2024 = pd.read_csv(templates_folder + master_applications_file_2024, parse_dates=[0], date_format="%d.%m.%Y")
        # contracts_dates_2023 = pd.read_csv(templates_folder + master_contracts_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
        # contracts_dates_2024 = pd.read_csv(templates_folder + master_contracts_file_2024, parse_dates=[0], date_format="%d.%m.%Y")
        leads_dates_2023 = pd.read_csv(templates_folder + master_leads_file_2023, parse_dates=[0], date_format="%d.%m.%Y")
        leads_dates_2024 = pd.read_csv(templates_folder + master_leads_file_2024, parse_dates=[0], date_format="%d.%m.%Y")
        leads_dates_2024_by_program = pd.read_excel(templates_folder + bitrix_file_2024, usecols="J:N") #, parse_dates=[0], date_format="%d.%m.%Y  %hh:%mm:%ss")
        leads_dates_2024_by_program['leads_dates'] = pd.to_datetime(leads_dates_2024_by_program['leads_dates'], errors='coerce', format="%d.%m.%Y  %hh:%mm:%ss")
        leads_dates_2024_by_program[col_programs_names].fillna(main_studyonline, inplace=True)


        asav_2023 = pd.read_excel(templates_folder + asav_file_2023, parse_dates=[0, 1], skiprows=1, date_format="%d.%m.%Y")
        asav_2023['applications_dates'] = pd.to_datetime(asav_2023['applications_dates'], format='%Y-%m-%d 00:00:00')
        asav_2023['contracts_dates'] = pd.to_datetime(asav_2023['contracts_dates'], errors='coerce', format='%d.%m.%Y')

        asav_2024 = pd.read_excel(templates_folder + asav_file_2024, parse_dates=[0, 1], skiprows=1, date_format="%d.%m.%Y")
        asav_2024['applications_dates'] = pd.to_datetime(asav_2024['applications_dates'], format='%Y-%m-%d 00:00:00')
        asav_2024['contracts_dates'] = pd.to_datetime(asav_2024['contracts_dates'], errors='coerce', format='%d.%m.%Y')

    except:
        print("Files of previous years are not founded or have errors")
        # , master_applications_file_2023)
        # print(master_contracts_file_2023)
        # print(master_applications_file_2024)
        # print(master_contracts_file_2024)
        print(master_leads_file_2023)
        print(master_leads_file_2024)
        print(bitrix_file_2024)
        print(asav_file_2023)
        print(asav_file_2024)


    now = datetime.now()
    delta_now_2023 = timedelta(days=365+366) # TODO check both deltas
    delta_now_2024 = timedelta(days=366)

    asav_2023_no_duplicates = asav_2023.drop_duplicates(subset=[col_id_asav])
    asav_2024_no_duplicates = asav_2024.drop_duplicates(subset=[col_id_asav])


    df_pivot = pd.DataFrame.from_dict({'leads' :
                                {2023: leads_dates_2023.where(leads_dates_2023 + delta_now_2023 <= now).count(),
                                 2024: leads_dates_2024.where(leads_dates_2024 + delta_now_2024 <= now).count()},
                                # 'applications_old' :
                                # {2023: applications_dates_2023.where(applications_dates_2023 + delta_now_2023 <= now).count(),
                                #  2024: applications_dates_2024.where(applications_dates_2024 + delta_now_2024 <= now).count()},
                                # 'contracts_old' :
                                # {2023: contracts_dates_2023.where(contracts_dates_2023 + delta_now_2023 <= now).count(),
                                #  2024: contracts_dates_2024.where(contracts_dates_2024 + delta_now_2024 <= now).count()},
                                'applications' :
                                {2023: asav_2023[asav_2023['applications_dates'] + delta_now_2023 <= now]['applications_dates'].count(),
                                 2024: asav_2024[asav_2024['applications_dates'] + delta_now_2024 <= now]['applications_dates'].count()},
                                'contracts' :
                                {2023: asav_2023[asav_2023['contracts_dates'] + delta_now_2023 <= now]['contracts_dates'].count(),
                                 2024: asav_2024[asav_2024['contracts_dates'] + delta_now_2024 <= now]['contracts_dates'].count()},
                                'applications_unique' :
                                {2023: asav_2023_no_duplicates[asav_2023_no_duplicates['applications_dates'] + delta_now_2023 <= now]['applications_dates'].count(),
                                 2024: asav_2024_no_duplicates[asav_2024_no_duplicates['applications_dates'] + delta_now_2024 <= now]['applications_dates'].count()}
                                }) #columns=['year', 'applications', 'contracts']
    df_leads_prev        = leads_dates_2024_by_program[leads_dates_2024_by_program['leads_dates'] + delta_now_2024 <= now].groupby(col_programs_names)[col_programs_names].count()
    df_applications_prev = asav_2024[asav_2024['applications_dates'] + delta_now_2024 <= now].groupby(master_col_programs)[master_col_programs].count()
    df_contracts_prev    = asav_2024[asav_2024['contracts_dates']    + delta_now_2024 <= now].groupby(master_col_programs)[master_col_programs].count()

    print("Исторические данные считаны")
    return df_pivot, df_leads_prev, df_applications_prev, df_contracts_prev

def process_foreign_programs(df, programs_names):
    df[master_foreign_col_programs_2] = df[master_foreign_col_programs_2].fillna("")
    is_online = df[master_foreign_col_programs_1].isin(programs_names)
    for i, row in df.iterrows():
        if not is_online.loc[i] or row[master_foreign_col_faculty_1] == "Факультет Санкт-Петербургская школа экономики и менеджмента":
            df.loc[i, master_foreign_col_programs_1] = df.loc[i, master_foreign_col_programs_2]

    # df[master_foreign_col_programs_1] = df[master_foreign_col_programs_1] if df[master_foreign_col_programs_1].isin(programs_names) and df[master_foreign_col_faculty_1] != "Факультет Санкт-Петербургская школа экономики и менеджмента" else df[master_foreign_col_programs_2]
    # df[master_foreign_col_programs_1].fillna(df[master_foreign_col_programs_2])
    df = df[df[master_foreign_col_programs_1].isin(programs_names)]
    # df[master_foreign_col_faculty_1] = df[master_foreign_col_faculty_1].fillna("")
    # df[master_foreign_col_faculty_2] = df[master_foreign_col_faculty_2].fillna("")
    # # df[master_foreign_col_program_final] = df[master_foreign_col_program_final].fillna("")
    # # df[master_foreign_col_faculty_final] = df[master_foreign_col_faculty_final].fillna("")
    # df[col_program] = df[master_foreign_col_programs_1] + df[master_foreign_col_programs_2]
    return df

def process_bitrix_by_week(df, df_bitrix_after_april):
    # programs = df[col_program]
    # df[col]
    # for program in programs:

    df_bitrix_after_april[bitrix_col_date] = pd.to_datetime(df_bitrix_after_april[bitrix_col_date]) - pd.to_timedelta(7, unit='d')
    df_bitrix_after_april = df_bitrix_after_april.groupby([col_program, pd.Grouper(key=bitrix_col_date, freq='W-MON')])[col_program] \
        .count() \
        .reset_index() \
        .sort_values(bitrix_col_date)
    print (df_bitrix_after_april)

    # bins = [0, 17, 23, 29, 35, 41, 47, float('inf')]
    # labels = ['0-17', '18-23', '24-29', '30-35', '36-41', '42-47', '48+']

    # # Используем pd.cut для разбиения на интервалы
    # categories = pd.cut(age_column, bins=bins, labels=labels, right=True, include_lowest=True)

    # # Считаем количество в каждом диапазоне
    # counts = categories.value_counts().sort_index()

    # return np.array2string(counts.values, separator=";")[1:-1]

    return df_bitrix_after_april

def process_current_files():

    NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents


    # папки и файлы для загрузки
    relative_folder = "data/"
    templates_folder = "templates/"

    programs_file = "programs.xlsx"
    template_file = "template.xlsx"

    # dashboard_file = "dashboard.xlsx"

    bitrix_file = "bitrix.xls"
    bitrix_file_before_april = "bitrix_2024-10-01_2025-03-31.xlsx"
    portal_file = "portal.xls"

    master_file = "asav.xlsx"
    master_file_foreign = "asav_foreign.xlsx"
    # master_file_sheet_name = "только онлайн"

    bachelor_app_file = "bac_applications.xls"
    bachelor_con_file = "bac_contracts.xls"
    bachelor_enr_file = "bac_enrolled.xlsx"

    # считывание файлов
    try:
        # cчитываем базу данных програм
        print("Начинаем считывать базу программ")
        df_online_programs = pd.read_excel(templates_folder + programs_file)
        df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
        df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
        df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
        print("База программ обработана")
    except:
        print("Потерялся " + programs_file + " - нужна база программ")
        return "Error program database"


    try:# Шаблон дашборда / template
        print("Начинаем считывать шаблон дашборда")
        # создаем дашборд по магистратурам добавляя туда программы из базы
        df_master_dashboard = pd.read_excel(templates_folder + template_file)
        df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
        df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
        df_master_dashboard = df_master_dashboard.fillna(0)
        df_master_dashboard[[col_plan_rus, col_plan_foreign]] = df_master_dashboard[[col_plan_rus, col_plan_foreign]].astype(int)

        df_bachelor_dashboard = pd.read_excel(templates_folder + template_file)
        df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
        df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
        df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
        df_bachelor_dashboard[[col_plan_rus, col_plan_foreign]] = df_bachelor_dashboard[[col_plan_rus, col_plan_foreign]].astype(int)
        print("Шаблон дашборда считан")
    except:
        print("Потерялся " + template_file + " - без него дашборд не собрать")
        return "Error dashboard template"

    try:# Число лидов со studyonline с 1 апреля по настоящее время. Почему-то это html таблица, хотя файл xls
        print("Начинаем считывать данные от Битрикса в html-формате")
        df_bitrix_after_april = pd.read_html(relative_folder + bitrix_file, header=0)[0]
        df_bitrix_after_april[col_programs_names].fillna(main_studyonline, inplace=True)
        print("Данные от Битрикса считаны")
        # pd.read_excel(relative_folder + bitrix_file)
    except:
        try:# Число лидов со studyonline с 1 апреля по настоящее время. На случай, если html чтение не сработало
            print("Начинаем считывать данные от Битрикса в xls-формате")
            df_bitrix_after_april = pd.read_excel(relative_folder + bitrix_file, header=0)
            df_bitrix_after_april[col_programs_names].fillna(main_studyonline, inplace=True)
            print("Данные от Битрикса считаны")
            # pd.read_excel(relative_folder + bitrix_file)
        except:
            print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
            df_bitrix_after_april = pd.DataFrame()



    try:# Число лидов c портала c 1 октября по настоящее время. Почему-то это html таблица, хотя файл xls
        print("Начинаем считывать данные от Портала")
        df_portal = pd.read_html(relative_folder + portal_file, header=0)[0]
        df_portal[col_programs_names].fillna(main_studyonline, inplace=True)
        print("Данные от Портала считаны")
        # pd.read_excel(relative_folder + bitrix_file)
    except:
        print("Нет выгрузки заявок с Портала или она называется не " + portal_file)
        df_portal = pd.DataFrame()

    try:# Число лидов из битрикс до 1 апреля (не включительно). Почему-то это html таблица, хотя файл xls
        print("Начинаем считывать данные от Битрикса до 31.03")
        df_bitrix_before_april = pd.read_excel(templates_folder + bitrix_file_before_april, usecols="I:N")
        df_bitrix_before_april[col_programs_names].fillna(main_studyonline, inplace=True)
        print("Данные от Битрикса до 31.03 считаны")
        # pd.read_excel(relative_folder + bitrix_file)
    except:
        print("Нет выгрузки заявок из Битрикса до 31.03 или она называется не " + bitrix_file_before_april)
        df_bitrix_before_april = pd.DataFrame()

    try:
        leads_after_april = df_bitrix_after_april.groupby(col_programs_names)[col_programs_names].count()
        leads_after_april = pd.DataFrame({'program_bitrix':leads_after_april.index, 'values':leads_after_april.values})
    except:
        leads_after_april = pd.DataFrame(columns=['program_bitrix', 'values'])

    try:
        leads_before_april = df_bitrix_before_april.groupby(col_programs_names)[col_programs_names].count()
        leads_before_april = pd.DataFrame({'program_bitrix':leads_before_april.index, 'values':leads_before_april.values})
    except:
        leads_before_april = pd.DataFrame(columns=['program_bitrix', 'values'])

    #     # достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
    # master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    # master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
    # df_master_dashboard[col_applications_since_april] = insert_values(df_master_dashboard, master_applications, col_program, col_applications_since_april)


    # try:
    #     df_master_2024 = pd.read_csv(templates_folder + master_file_before_april, usecols="J:T")
    #     master_applications_2024 = df_master_2024.groupby(master_col_programs)[master_col_programs].count()
    #     master_applications_2024 = pd.DataFrame({col_program:master_applications_2024.index, 'values':master_applications_2024.values})
    #     df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications_2024, col_program, col_applications)
    # except:
    #     print("Ошибка при чтении файла asav_2024-10-01_2025-03-31.csv")
    #     df_master_dashboard[col_applications] = 0


    try:
        leads_portal = df_portal.groupby(col_programs_names)[col_programs_names].count()
        leads_portal = pd.DataFrame({'program_bitrix':leads_portal.index, 'values':leads_portal.values})
    except:
        leads_portal = pd.DataFrame(columns=['program_bitrix', 'values'])


    df_master_dashboard  [col_leads] = insert_values(df_master_dashboard,   leads_before_april, 'program_bitrix', col_leads)
    df_bachelor_dashboard[col_leads] = insert_values(df_bachelor_dashboard, leads_before_april, 'program_bitrix', col_leads)

    df_master_dashboard  [col_leads_after_april] = insert_values(df_master_dashboard,   leads_after_april, 'program_bitrix', col_leads_after_april)
    df_bachelor_dashboard[col_leads_after_april] = insert_values(df_bachelor_dashboard, leads_after_april, 'program_bitrix', col_leads_after_april)

    df_master_dashboard  [col_leads] += df_master_dashboard  [col_leads_after_april] # TODO check
    df_bachelor_dashboard[col_leads] += df_bachelor_dashboard[col_leads_after_april]

    main_leads             = leads_before_april.loc[leads_before_april['program_bitrix'] == main_studyonline, 'values'].values[0]
    main_leads_after_april = leads_after_april.loc[leads_after_april['program_bitrix'] == main_studyonline, 'values'].values[0]
    main_leads            += main_leads_after_april # TODO check

    df_master_dashboard  [col_leads_partners] = insert_values(df_master_dashboard,   leads_portal, 'program_bitrix', col_leads_partners)
    df_bachelor_dashboard[col_leads_partners] = insert_values(df_bachelor_dashboard, leads_portal, 'program_bitrix', col_leads_partners)
    # main_leads_portal = leads_portal.loc[leads_portal['program_bitrix'] == main_studyonline, 'values'].values[0]


    # АСАВ иностранцы
    try:
        print("Начинаем считывать данные от АСАВ по иностранцам")
        df_master_foreign = pd.read_excel(relative_folder + master_file_foreign, skiprows=1, usecols="F:BJ") #sheet_name=master_file_sheet_name,
        print("Данные от АСАВ по иностранцам считаны")
    except:
        print("Ошибка в обработке АСАВ по иностранцам, возможно нет выгрузки из АСАВ или она называется не " + master_file_foreign)
        df_master_foreign = pd.DataFrame(columns=[master_col_programs, master_foreign_col_contracts, master_foreign_col_payments, master_foreign_col_enrollments])


    df_master_foreign = process_foreign_programs(df_master_foreign, df_online_programs[col_program])

    master_applications_foreign = df_master_foreign.groupby(master_foreign_col_programs_1)[master_foreign_col_programs_1].count()
    master_applications_foreign = pd.DataFrame({col_program:master_applications_foreign.index, 'values':master_applications_foreign.values})
    df_master_dashboard[col_applications_foreign] = insert_values(df_master_dashboard, master_applications_foreign, col_program, col_applications_foreign)

    master_contracts_foreign = df_master_foreign[df_master_foreign[master_foreign_col_contracts] == "Да"].groupby(master_foreign_col_programs_1)[master_foreign_col_programs_1].count()
    master_contracts_foreign = pd.DataFrame({col_program:master_contracts_foreign.index, 'values':master_contracts_foreign.values})
    df_master_dashboard[col_contracts_foreign] = insert_values(df_master_dashboard, master_contracts_foreign, col_program, col_contracts_foreign)

    master_payments_foreign = df_master_foreign[df_master_foreign[master_foreign_col_payments] == "Да"].groupby(master_foreign_col_programs_1)[master_foreign_col_programs_1].count()
    master_payments_foreign = pd.DataFrame({col_program:master_payments_foreign.index, 'values':master_payments_foreign.values})
    df_master_dashboard[col_payments_foreign] = insert_values(df_master_dashboard, master_payments_foreign, col_program, col_payments_foreign)


#     master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
#     master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
#     df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

#     master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
#     master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
#     df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

#     master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
#     master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
#     df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

#     master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
#     master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
#     df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)

#     master_male = df_master[df_master[col_gender_asav] == "Муж."].groupby(master_col_programs)[master_col_programs].count()
#     master_male = pd.DataFrame({col_program:master_male.index, 'values':master_male.values})
#     df_master_dashboard[col_male] = insert_values(df_master_dashboard, master_male, col_program, col_male)

#     master_female = df_master[df_master[col_gender_asav] == "Жен."].groupby(master_col_programs)[master_col_programs].count()
#     master_female = pd.DataFrame({col_program:master_female.index, 'values':master_female.values})
#     df_master_dashboard[col_female] = insert_values(df_master_dashboard, master_female, col_program, col_female)


#     df_master[col_birthday] = pd.to_datetime(df_master[col_birthday]).apply(num_years)
#     master_years_bars = df_master.groupby(master_col_programs)[col_birthday].apply(categorize_ages)
#     master_years_bars = pd.DataFrame({col_program:master_years_bars.index, 'values':master_years_bars.values})
#     df_master_dashboard[col_ages] = insert_values(df_master_dashboard, master_years_bars, col_program, col_ages)

#     master_years_mean = df_master.groupby(master_col_programs)[col_birthday].mean()
#     master_years_mean = pd.DataFrame({col_program:master_years_mean.index, 'values':master_years_mean.values})
#     df_master_dashboard[col_ages_mean] = insert_values(df_master_dashboard, master_years_mean, col_program, col_ages_mean)

# №№№№№№№№№№№


    # АСАВ
    try:
        print("Начинаем считывать данные от АСАВ")
        df_master = pd.read_excel(relative_folder + master_file, skiprows=1, usecols="A:DT") #sheet_name=master_file_sheet_name,
        print("Данные от АСАВ считаны")
    except:
        print("Ошибка в обработке АСАВ, возможно нет выгрузки из АСАВ или она называется не " + master_file)
        df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

    # убираем офлайн-психов TODO - проверить международный бизнес и другие программы с треками
    df_master['Магистерская специализация'] = df_master['Магистерская специализация'].fillna('')
    df_master = df_master[~df_master['Магистерская специализация'].str.contains("офлайн")]
    df_master = df_master.dropna(subset=[col_birthday])


    # достаем данные по ЛК, договорам, оплатам и зачислениям из АСАВ
    master_applications = df_master.groupby(master_col_programs)[master_col_programs].count() #.rename("program")#.sort_values(ascending=False)
    master_applications = pd.DataFrame({col_program:master_applications.index, 'values':master_applications.values})
    df_master_dashboard[col_applications] = insert_values(df_master_dashboard, master_applications, col_program, col_applications)

    master_contracts = df_master[df_master[master_col_contracts].notna()].groupby(master_col_programs)[master_col_programs].count()
    master_contracts = pd.DataFrame({col_program:master_contracts.index, 'values':master_contracts.values})
    df_master_dashboard[col_contracts] = insert_values(df_master_dashboard, master_contracts, col_program, col_contracts)

    master_payments = df_master[df_master[master_col_payments] == "Оплачено"].groupby(master_col_programs)[master_col_programs].count()
    master_payments = pd.DataFrame({col_program:master_payments.index, 'values':master_payments.values})
    df_master_dashboard[col_payments] = insert_values(df_master_dashboard, master_payments, col_program, col_payments)

    master_enrollments = df_master[df_master[master_col_enrollments].notna()].groupby(master_col_programs)[master_col_programs].count()
    master_enrollments = pd.DataFrame({col_program:master_enrollments.index, 'values':master_enrollments.values})
    df_master_dashboard[col_enrollments] = insert_values(df_master_dashboard, master_enrollments, col_program, col_enrollments)

    master_male = df_master[df_master[col_gender_asav] == "Муж."].groupby(master_col_programs)[master_col_programs].count()
    master_male = pd.DataFrame({col_program:master_male.index, 'values':master_male.values})
    df_master_dashboard[col_male] = insert_values(df_master_dashboard, master_male, col_program, col_male)

    master_female = df_master[df_master[col_gender_asav] == "Жен."].groupby(master_col_programs)[master_col_programs].count()
    master_female = pd.DataFrame({col_program:master_female.index, 'values':master_female.values})
    df_master_dashboard[col_female] = insert_values(df_master_dashboard, master_female, col_program, col_female)


    df_master[col_birthday] = pd.to_datetime(df_master[col_birthday]).apply(num_years)
    master_years_bars = df_master.groupby(master_col_programs)[col_birthday].apply(categorize_ages)
    master_years_bars = pd.DataFrame({col_program:master_years_bars.index, 'values':master_years_bars.values})
    df_master_dashboard[col_ages] = insert_values(df_master_dashboard, master_years_bars, col_program, col_ages)

    master_years_mean = df_master.groupby(master_col_programs)[col_birthday].mean()
    master_years_mean = pd.DataFrame({col_program:master_years_mean.index, 'values':master_years_mean.values})
    df_master_dashboard[col_ages_mean] = insert_values(df_master_dashboard, master_years_mean, col_program, col_ages_mean)


    # АИС ПК
    try:
        print("Начинаем считывать данные от АИС ПК")
        df_bachelor_app = pd.read_excel(relative_folder + bachelor_app_file, usecols="I:Z") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        df_bachelor_con = pd.read_excel(relative_folder + bachelor_con_file, usecols="H:T") #, sheet_name=master_file_sheet_name, skiprows=1, usecols="L:DT")
        df_bachelor_enr = pd.read_excel(relative_folder + bachelor_enr_file, usecols="E:H") #, sheet_name=master_file_sheet_name, skiprows=1)
        print("Данные от АИС ПК считаны")

        bachelor_dict = {
    'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации - онлайн (О К)'        ,
    'Глобальные цифровые коммуникации (онлайн)'                        :'Глобальные цифровые коммуникации (Медиа) - онлайн (О К)',
    'Компьютерные науки и анализ данных (онлайн)'                      :'Компьютерные науки и анализ данных - онлайн (О К)'      ,
    'Экономический анализ (онлайн)'                                    :'Экономический анализ - онлайн (О К)'                    ,
    'Дизайн  (онлайн)'                                                 :'Дизайн - онлайн (О К)'                                  ,
    'Программные системы и автоматизация процессов разработки (онлайн)':'Программные системы и автоматизация процессов разработки - онлайн (О К)'
        }
         # достаем данные по ЛК, договорам, оплатам и зачислениям из АИС ПК
        bachelor_applications = df_bachelor_app.groupby(bachelor_col_programs)[bachelor_col_programs].count() #.rename("program")#.sort_values(ascending=False)
        bachelor_applications = pd.DataFrame({col_program:bachelor_applications.index, 'values':bachelor_applications.values})
        df_bachelor_dashboard[col_applications] = insert_values(df_bachelor_dashboard, bachelor_applications, col_program, col_applications)

        bachelor_contracts = df_bachelor_con.groupby(col_programs_names)[col_programs_names].count()
        bachelor_contracts = bachelor_contracts.rename(index=bachelor_dict)
        bachelor_contracts = pd.DataFrame({col_program:bachelor_contracts.index, 'values':bachelor_contracts.values})
        df_bachelor_dashboard[col_contracts] = insert_values(df_bachelor_dashboard, bachelor_contracts, col_program, col_contracts)

        bachelor_payments = df_bachelor_con[(df_bachelor_con[bachelor_col_payments] == "Оплачен")|(df_bachelor_con[bachelor_col_payments] == "Оплачен по квитанциям")].groupby(col_programs_names)[col_programs_names].count()
        bachelor_payments = bachelor_payments.rename(index=bachelor_dict)
        bachelor_payments = pd.DataFrame({col_program:bachelor_payments.index, 'values':bachelor_payments.values})
        df_bachelor_dashboard[col_payments] = insert_values(df_bachelor_dashboard, bachelor_payments, col_program, col_payments)

        bachelor_enrollments = df_bachelor_enr.groupby(bachelor_col_enrollments)[bachelor_col_enrollments].count()
        bachelor_enrollments = pd.DataFrame({col_program:bachelor_enrollments.index, 'values':bachelor_enrollments.values})
        df_bachelor_dashboard[col_enrollments] = insert_values(df_bachelor_dashboard, bachelor_enrollments, col_program, col_enrollments)

    except:
        print("Ошибка в обработке АИС ПК, возможно нет выгрузки из АИС ПК или она называется не:\n" + bachelor_app_file)
        print(bachelor_con_file)
        print(bachelor_enr_file)
        # df_master = pd.DataFrame(columns=[master_col_programs, master_col_contracts, master_col_payments, master_col_enrollments])

    df_history, df_leads_prev, df_applications_prev, df_contracts_prev = process_history_files() # Only for master for now

    df_applications_prev = pd.DataFrame({col_program:df_applications_prev.index, 'values':df_applications_prev.values})
    df_master_dashboard[col_applications_prev] = insert_values(df_master_dashboard, df_applications_prev, col_program, col_applications_prev)

    df_contracts_prev = pd.DataFrame({col_program:df_contracts_prev.index, 'values':df_contracts_prev.values})
    df_master_dashboard[col_contracts_prev] = insert_values(df_master_dashboard, df_contracts_prev, col_program, col_contracts_prev)

    asav_2025_no_duplicates = df_master.drop_duplicates(subset=['Unnamed: 0']) # TODO rename to col_id_asav

    df_history.loc[2025, 'applications_unique'] = asav_2025_no_duplicates[master_col_programs].count()

    df_leads_prev = pd.DataFrame({col_program_bitrix:df_leads_prev.index, 'values':df_leads_prev.values})
    main_leads_after_april_prev = df_leads_prev[df_leads_prev[col_program_bitrix] == main_studyonline]['values'].values[0]

    df_main_dashboard = pd.DataFrame(columns=df_master_dashboard.columns)
    df_main_dashboard.loc[len(df_main_dashboard)] = {col_program: main_studyonline, col_program_bitrix: main_studyonline, col_leads: main_leads, col_leads_after_april: main_leads_after_april, col_leads_after_april_prev: main_leads_after_april_prev}
    df = pd.concat([df_main_dashboard, df_master_dashboard, df_bachelor_dashboard], ignore_index=True, sort=False)

    df[col_leads_after_april_by_week] = process_bitrix_by_week(df, df_bitrix_after_april)

    df[col_leads_after_april_prev] = insert_values(df, df_leads_prev, col_program_bitrix, col_leads_after_april_prev)
    df = df.drop(columns=['program_bitrix'])

    df.fillna(0, inplace=True)

    # считаем второстепенные столбцы
    df[col_leads_total]                            = df[col_leads_partners] + df[col_leads]
    df[col_conversion_leads_to_contracts]          = df[col_contracts] / df[col_leads_total]
    df[col_needed_applications]              = round(df[col_plan_rus]/ NEEDED_APPLICATIONS_RATIO)
    df[col_conversion_applications_to_contracts]   = df[col_contracts] / df[col_applications]
    df[col_conversion_contracts_to_payments]       = df[col_payments]  / df[col_contracts]
    df[col_conversion_contracts_to_enrollments]    = df[col_enrollments]  / df[col_contracts]
    df[col_payments_div_plan_rus]                  = df[col_payments]  / df[col_plan_rus]
    df[col_payments_div_plan_foreign]              = df[col_payments_foreign]  / df[col_plan_foreign]
    df[col_income_1year     ] = df['price'] * df[col_payments] / 1000 # from thousands to millions
    df.loc[df['level'] == 'master', col_income_all]   = df[col_income_1year] * 2
    df.loc[df['level'] == 'bachelor', col_income_all] = df[col_income_1year] * 4
    # df[col_income_all       ] = df[col_income_1year]  * (2 if df['level'] == 'master' else 4) # TODO check later
    df[col_income_1year_hse ] = df[col_income_1year] * df['income_percent'] / 100
    df[col_income_all_hse   ] = df[col_income_all] * df['income_percent'] / 100

    # def cells_from_row(row):
    #     return "I" + str(row.name+2) + ":J" + str(row.name+2)

    # def gender_bars_function(row):
    #     return "=SPARKLINE(" + cells_from_row(row) + ',{"charttype","bar";"max",MAX(SUM(' + cells_from_row(row) + '),1);"color1","blue";"color2","red"})'
    # df[col_gender] = df.apply(gender_bars_function, axis=1)

    # def ages_bars_function(row):
    #     return "=sparkline(SPLIT(L"+ str(row.name + 2) + ',";"),{"charttype", "column"; "color", "gray"})'
    # df[col_ages_bars] = df.apply(ages_bars_function, axis=1)

    # =SPARKLINE(I3:J3,{"charttype","bar";"max",MAX(SUM(I3:J3),1);"color1","blue";"color2","red"})
    #=sparkline(SPLIT(L3,";"),{"charttype", "column"; "color", "gray"})

    df.replace(np.inf, 0, inplace=True)
    df.fillna(0, inplace=True)

    return df, df_history

In [ ]:

NEEDED_APPLICATIONS_RATIO = 30 / 100 #percents


# папки и файлы для загрузки
relative_folder = "../data/"
templates_folder = "../templates/"

programs_file = "programs.xlsx"
template_file = "template.xlsx"

# dashboard_file = "dashboard.xlsx"

bitrix_file = "bitrix.xls"
bitrix_file_before_april = "bitrix_2024-10-01_2025-03-31.xlsx"
portal_file = "portal.xls"

master_file = "asav.xlsx"
master_file_foreign = "asav_foreign.xlsx"
# master_file_sheet_name = "только онлайн"

bachelor_app_file = "bac_applications.xls"
bachelor_con_file = "bac_contracts.xls"
bachelor_enr_file = "bac_enrolled.xlsx"

# считывание файлов
try:
    # cчитываем базу данных програм
    print("Начинаем считывать базу программ")
    df_online_programs = pd.read_excel(templates_folder + programs_file)
    df_online_programs = df_online_programs[df_online_programs['format'] != 'offline'].reset_index(drop=True)
    df_online_master_programs = df_online_programs[df_online_programs['level'] == 'master'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    df_online_bachelor_programs = df_online_programs[df_online_programs['level'] == 'bachelor'].drop(columns=["format"]).sort_values(by=col_program).reset_index(drop=True)
    print("База программ обработана")
except:
    print("Потерялся " + programs_file + " - нужна база программ")
    print( "Error program database")


try:# Шаблон дашборда / template
    print("Начинаем считывать шаблон дашборда")
    # создаем дашборд по магистратурам добавляя туда программы из базы
    df_master_dashboard = pd.read_excel(templates_folder + template_file)
    df_master_dashboard = pd.concat([df_online_master_programs, df_master_dashboard])
    df_master_dashboard['program_bitrix'] = df_master_dashboard['program_bitrix'].fillna("")
    df_master_dashboard = df_master_dashboard.fillna(0)
    df_master_dashboard[[col_plan_rus, col_plan_foreign]] = df_master_dashboard[[col_plan_rus, col_plan_foreign]].astype(int)

    df_bachelor_dashboard = pd.read_excel(templates_folder + template_file)
    df_bachelor_dashboard = pd.concat([df_online_bachelor_programs, df_bachelor_dashboard])
    df_bachelor_dashboard['program_bitrix'] = df_bachelor_dashboard['program_bitrix'].fillna("")
    df_bachelor_dashboard = df_bachelor_dashboard.fillna(0)
    df_bachelor_dashboard[[col_plan_rus, col_plan_foreign]] = df_bachelor_dashboard[[col_plan_rus, col_plan_foreign]].astype(int)
    print("Шаблон дашборда считан")
except:
    print("Потерялся " + template_file + " - без него дашборд не собрать")
    print( "Error template")

try:# Число лидов со studyonline с 1 апреля по настоящее время. Почему-то это html таблица, хотя файл xls
    print("Начинаем считывать данные от Битрикса в html-формате")
    df_bitrix_after_april = pd.read_html(relative_folder + bitrix_file, header=0)[0]
    df_bitrix_after_april[col_programs_names].fillna(main_studyonline, inplace=True)
    print("Данные от Битрикса считаны")
    # pd.read_excel(relative_folder + bitrix_file)
except:
    try:# Число лидов со studyonline с 1 апреля по настоящее время. На случай, если html чтение не сработало
        print("Начинаем считывать данные от Битрикса в xls-формате")
        df_bitrix_after_april = pd.read_excel(relative_folder + bitrix_file, header=0)
        df_bitrix_after_april[col_programs_names].fillna(main_studyonline, inplace=True)
        print("Данные от Битрикса считаны")
        # pd.read_excel(relative_folder + bitrix_file)
    except:
        print("Нет выгрузки из Битрикса или она называется не " + bitrix_file)
        df_bitrix_after_april = pd.DataFrame()


In [ ]:
pd.to_datetime(df_bitrix_after_april[bitrix_col_date])

In [ ]:
df_bitrix_after_april.groupby(col_programs_names)[col_programs_names].count()

In [ ]:
# df_bitrix_after_april[bitrix_col_date] = pd.to_datetime(df_bitrix_after_april[bitrix_col_date]) - pd.to_timedelta(7, unit='d')

df_bitrix_after_april.groupby([col_programs_names, pd.Grouper(key=bitrix_col_date, freq='W-MON')])[bitrix_col_date] \
        .count() \
        .reset_index() \
        .sort_values(bitrix_col_date)


In [ ]:
def process_bitrix_by_week(df, df_bitrix_after_april):
    # programs = df[col_program]
    # df[col]
    # for program in programs:

    df_bitrix_after_april[bitrix_col_date] = pd.to_datetime(df_bitrix_after_april[bitrix_col_date]) - pd.to_timedelta(7, unit='d')
    df_bitrix_after_april = df_bitrix_after_april.groupby([col_program, pd.Grouper(key=bitrix_col_date, freq='W-MON')])[col_program] \
        .count() \
        .reset_index() \
        .sort_values(bitrix_col_date)
    print (df_bitrix_after_april)

    # bins = [0, 17, 23, 29, 35, 41, 47, float('inf')]
    # labels = ['0-17', '18-23', '24-29', '30-35', '36-41', '42-47', '48+']

    # # Используем pd.cut для разбиения на интервалы
    # categories = pd.cut(age_column, bins=bins, labels=labels, right=True, include_lowest=True)

    # # Считаем количество в каждом диапазоне
    # counts = categories.value_counts().sort_index()

    # return np.array2string(counts.values, separator=";")[1:-1]

    return df_bitrix_after_april

In [ ]:
df = df_bitrix_after_april.copy()
df[bitrix_col_date] = pd.to_datetime(df[bitrix_col_date])

# Вычисляем номер недели (можно также использовать понедельник недели как якорь)
df['week_start'] = df[bitrix_col_date].dt.to_period('W-MON').apply(lambda r: r.start_time)

# Группируем по программе и неделе
weekly_counts = df.groupby([col_programs_names, 'week_start']).size().reset_index(name='count')

# Получим все уникальные программы и все недели
all_programs = weekly_counts[col_programs_names].unique()
all_weeks = pd.date_range(start=pd.Timestamp(year=2025, month=4, day=1, hour=0, minute=0, second=0),
                          end=weekly_counts['week_start'].max(),
                          freq='W-TUE')  # каждую неделю по понедельникам

all_weeks

In [ ]:
df = df_bitrix_after_april.copy()
df[bitrix_col_date] = pd.to_datetime(df[bitrix_col_date])

# Вычисляем номер недели (можно также использовать понедельник недели как якорь)
df['week_start'] = df[bitrix_col_date].dt.to_period('W-MON').apply(lambda r: r.start_time)

# Группируем по программе и неделе
weekly_counts = df.groupby([col_programs_names, 'week_start']).size().reset_index(name='count')

# Получим все уникальные программы и все недели
all_programs = weekly_counts[col_programs_names].unique()
all_weeks = pd.date_range(start=pd.Timestamp(year=2025, month=4, day=1, hour=0, minute=0, second=0),
                          end=weekly_counts['week_start'].max(),
                          freq='W-TUE')  # каждую неделю по понедельникам

# Создаем полную сетку: программа × неделя
full_index = pd.MultiIndex.from_product([all_programs, all_weeks], names=[col_programs_names, 'week_start'])
full_df = pd.DataFrame(index=full_index).reset_index()

# Объединяем с посчитанными заявками
merged = pd.merge(full_df, weekly_counts, how='left', on=[col_programs_names, 'week_start'])
merged['count'] = merged['count'].fillna(0).astype(int)

# Группируем по программе и объединяем значения в строку через ";"
result = merged.groupby(col_programs_names)['count'].apply(lambda x: ';'.join(map(str, x))).reset_index()

# Результат в DataFrame `result`
display(result)

# Stage 11
18.06.25

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("../data/asav.xlsx", skiprows=1, usecols="A:AB, CY:DW")
df = df.dropna(how='all', ignore_index=True)
#pd.to_datetime(df[df.columns[-1]], errors='raise')
display(df.loc[700:720]) #[df.loc[518:601, df.columns[-1]] != '2025-05-20 00:00:00'])

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
display(df)

In [ ]:
!python --version